<a href="https://colab.research.google.com/github/SampMark/ETL-de-dados-da-PNP/blob/main/web_scraping_mapa_RFEPT/GitHub_Web_Scraping_and_merge_RFEPT_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==============================================================================
# @title **SEÇÃO 1: IMPORTAÇÕES E CONFIGURAÇÕES GLOBAIS**
# ==============================================================================

# Instala dependências
!pip install unidecode --quiet

import requests
from bs4 import BeautifulSoup, Tag
import pandas as pd
import numpy as np
import re
import time
import logging
from typing import List, Dict, Tuple, Optional, Any
import unicodedata
from unidecode import unidecode
from requests.exceptions import ReadTimeout, ConnectionError, HTTPError, RequestException
print("Dependências e bibliotecas instaladas com sucesso!")

# Autenticação no Google Sheets
from google.colab import auth
import gspread
import google.auth
from gspread import Client
print("Importação de bibliotecas concluída.")

# --- Configuração do Logging ---
# Configura um sistema de logging para registrar informações da execução.
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Autentica e inicializa o cliente gspread
# Isso deve ser feito uma vez antes de qualquer operação de planilha.
auth.authenticate_user()
credentials, project = google.auth.default()
gc = Client(auth=credentials)
print("Autenticação e cliente gspread inicializados.")

# --- Constantes Globais ---
# URL base do portal do MEC para a Rede Federal. O '{uf}' será substituído
# dinamicamente pela sigla de cada estado.
URL_BASE = "https://www.gov.br/mec/pt-br/assuntos/ept/rede-federal/{uf}"

# Lista das 27 Unidades Federativas do Brasil, formatadas para a URL.
# Esta lista garante a cobertura completa do território nacional.
UFS = [
    "minas-gerais", "rio-de-janeiro", "pernambuco", "acre", "alagoas", "amapa", "amazonas", "bahia", "ceara",
    "distrito-federal", "espirito-santo", "goias", "maranhao",
    "mato-grosso", "mato-grosso-do-sul", "para",
    "paraiba", "parana",  "piaui",
    "rio-grande-do-norte", "rio-grande-do-sul", "rondonia", "roraima",
    "santa-catarina", "sao-paulo", "sergipe", "tocantins"
]

# Cabeçalho da requisição para simular um navegador comum. Isso aumenta a
# chance de sucesso da requisição, evitando bloqueios por firewalls de
# aplicações web que filtram bots simples.
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Dicionário para mapear UF para Região, útil para preencher a coluna 'Região'
# para as novas unidades adicionadas que não estavam no dataset original.
MAPA_REGIOES = {
    'AC': 'Norte', 'AP': 'Norte', 'AM': 'Norte', 'PA': 'Norte', 'RO': 'Norte', 'RR': 'Norte', 'TO': 'Norte',
    'AL': 'Nordeste', 'BA': 'Nordeste', 'CE': 'Nordeste', 'MA': 'Nordeste', 'PB': 'Nordeste', 'PE': 'Nordeste', 'PI': 'Nordeste', 'RN': 'Nordeste', 'SE': 'Nordeste',
    'DF': 'Centro-Oeste', 'GO': 'Centro-Oeste', 'MT': 'Centro-Oeste', 'MS': 'Centro-Oeste',
    'ES': 'Sudeste', 'MG': 'Sudeste', 'RJ': 'Sudeste', 'SP': 'Sudeste',
    'PR': 'Sul', 'RS': 'Sul', 'SC': 'Sul'
}

# Mapa de slug→sigla
SLUG_TO_UF = {
    "acre": "AC", "alagoas": "AL", "amapa": "AP", "amazonas": "AM",
    "bahia": "BA", "ceara": "CE", "distrito-federal": "DF",
    "espirito-santo": "ES", "goias": "GO", "maranhao": "MA",
    "mato-grosso": "MT", "mato-grosso-do-sul": "MS", "minas-gerais": "MG",
    "para": "PA", "paraiba": "PB", "parana": "PR", "pernambuco": "PE",
    "piaui": "PI", "rio-de-janeiro": "RJ", "rio-grande-do-norte": "RN",
    "rio-grande-do-sul": "RS", "rondonia": "RO", "roraima": "RR",
    "santa-catarina": "SC", "sao-paulo": "SP", "sergipe": "SE", "tocantins": "TO",
}

# Mapa para consolidação de regras de detecção de cabeçalho
CAMPUS_PREFIXES = (
    'CAMPUS', 'CÂMPUS', 'CAMPUS AVANÇADO',
    'CENTRO DE REFERÊNCIA', 'CENTRO DE EDUCAÇÃO PROFISSIONAL',
    'COLÉGIO TÉCNICO', 'ESCOLA TÉCNICA', 'UNIDADE', 'POLO',
    'POLO DE INOVAÇÃO', 'UNED', 'CENTRO DE ENSINO'
)

# Mapa de renomeação
MAPA_CORRECAO = {
    'CEFET-MG - Centro Federal de Educação Tecnológica de Minas Gerais': 'Centro Federal de Educação Tecnológica de Minas Gerais',
    'Instituto Federal Sudeste de Minas Gerais': 'Instituto Federal do Sudeste de Minas Gerais',
    'Instituto Federal do Mato Grosso do Sul': 'Instituto Federal de Mato Grosso do Sul',
    'Instituto Federal do Mato Grosso': 'Instituto Federal de Mato Grosso'
    # Adicione outras regras de renomeação aqui conforme necessário
}

# Mapa de renomeação para Campus_IF
MAPA_CORRECAO_CAMPUS = {
    r'^Unidade\b': 'Uned',
    r'^Unidade Maracanã\b': 'Uned Maracanã',
    r'^Unidade Belo Horizonte\b': 'Uned Belo Horizonte',
    'Campus Avançado Pecém': 'Campus Pecém',
    'Campus Santa Inês – Ifma': 'Campus Santa Inês',
    'Centro de Referência em Educação Infantil': 'Centro de Referência em Educação Infantil Realengo (CREIR)',
    'Reitoria do Colégio Agrícola': 'Colégio Agrícola',
    'Reitoria do Colégio Técnico': 'Colégio Técnico',
    'Reitoria do Colégio Politécnico': 'Colégio Politécnico',
    'Reitoria do Escola': 'Escola',
    'Reitoria do Centro': 'Centro',
    # 'Reitoria do Centro': 'Centro de Educação',
    # 'Reitoria do Centro Técnico': 'Centro Técnico',
    'Reitoria do CEFET-MG - Centro Federal de Educação Tecnológica de Minas Gerais': 'Reitoria do Centro Federal de Educação Tecnológica de Minas Gerais',
    'Reitoria do Universidade': 'Reitoria da Universidade',
    'Uned Belo Horizonte (Campus Nova Suíça)': 'Uned Belo Horizonte (Nova Suíça)',
    'Uned Belo Horizonte (Campus Nova Gameleira)': 'Uned Belo Horizonte (Nova Gameleira)',
    'Uned Belo Horizonte (Campus Gameleira)': 'Uned Belo Horizonte (Gameleira)'

}

# Dicionário onde a CHAVE é o nome-base da instituição e o VALOR é a sigla desejada.
MAPA_SIGLAS_IFS = {
    "Instituto Federal do Acre": "IFAC",
    "Instituto Federal de Alagoas": "IFAL",
    "Instituto Federal do Amazonas": "IFAM",
    "Instituto Federal do Amapá": "IFAP",
    "Instituto Federal Baiano": "IF Baiano",
    "Instituto Federal da Bahia": "IFBA",
    "Instituto Federal do Ceará": "IFCE",
    "Instituto Federal de Brasília": "IFB",
    "Instituto Federal do Espírito Santo": "IFES",
    "Instituto Federal Goiano": "IF Goiano",
    "Instituto Federal de Goiás": "IFG",
    "Instituto Federal do Maranhão": "IFMA",
    "Centro Federal de Educação Tecnológica de Minas Gerais": "CEFET-MG",
    "Instituto Federal de Minas Gerais": "IFMG",
    "Instituto Federal do Norte de Minas Gerais": "IFNMG",
    "Instituto Federal do Sudeste de Minas Gerais": "IF Sudeste MG",
    "Instituto Federal do Sul de Minas Gerais": "IFSULDEMINAS",
    "Instituto Federal do Triângulo Mineiro": "IFTM",
    "Instituto Federal de Mato Grosso do Sul": "IFMS",
    "Instituto Federal de Mato Grosso": "IFMT",
    "Instituto Federal do Pará": "IFPA",
    "Instituto Federal da Paraíba": "IFPB",
    "Instituto Federal de Pernambuco": "IFPE",
    "Instituto Federal do Sertão Pernambucano": "IF Sertão-PE",
    "Instituto Federal do Piauí": "IFPI",
    "Instituto Federal do Paraná": "IFPR",
    "Centro Federal de Educação Tecnológica Celso Suckow da Fonseca": "CEFET-RJ",
    "Colégio Pedro II": "CPII",
    "Instituto Federal Fluminense": "IFF",
    "Instituto Federal do Rio de Janeiro": "IFRJ",
    "Instituto Federal do Rio Grande do Norte": "IFRN",
    "Instituto Federal de Rondonia": "IFRO",
    "Instituto Federal de Rondônia": "IFRO",
    "Instituto Federal de Roraima": "IFRR",
    "Instituto Federal Farroupilha": "IF Farroupilha",
    "Instituto Federal Sul-rio-grandense": "IFSul",
    "Instituto Federal do Rio Grande do Sul": "IFRS",
    "Instituto Federal Catarinense": "IFC",
    "Instituto Federal de Santa Catarina": "IFSC",
    "Instituto Federal de Sergipe": "IFS",
    "Instituto Federal de São Paulo": "IFSP",
    "Instituto Federal do Tocantins": "IFTO",
    "Instituto Federal de Tocantins": "IFTO", # Escrita errada 'de'
    "Universidade Tecnológica Federal do Paraná": "UTFPR",

    # Casos de Escolas/Colégios vinculados a Universidades
    "Centro de Educação Profissional – Universidade Federal do Triângulo Mineiro (Uftm)": "CEFORES/UFMT",
    "Centro de Ensino e Desenvolvimento Agrário – Universidade Federal de Viçosa (Ufv)": "CEDAF/UFV",
    "Centro Técnico Pedagógico – Universidade Federal de Minas Gerais (Ufmg)": "CP/UFMG",
    "Colégio Agrícola de Bom Jesus - Universidade Federal do Piauí (Ufpi)": "CAJB/UFPI",
    "Colégio Agrícola Dom Agostinho Ikas – Universidade Federal Rural do Pernambuco (Ufrpe)": "CODAI/UFRPE",
    "Colégio Agrícola Vidal de Negreiros – Universidade Federal da Paraíba (Ufpb)": "CAVN/UFPB",
    "Colégio Politécnico de Santa Maria - Universidade Federal de Santa Maria (Ufsm)": "CPT/UFSM",
    "Colégio Técnico – Universidade Federal de Minas Gerais (Ufmg)": "COLTEC/UFMG",
    "Colégio Técnico – Universidade Federal Rural do Rio de Janeiro (Ufrrj)": "CTUR/UFRRJ",
    "Colégio Técnico de Bom Jesus - Universidade Federal do Piauí (Ufpi)": "CTBJ/UFPI",
    "Colégio Técnico de Floriano – Universidade Federal do Piauí (Ufpi)": "CTF/UFPI",
    "Colégio Técnico de Teresina - Universidade Federal do Piauí (Ufpi)": "CTT/UFPI",
    "Colégio Técnico Industrial Santa Maria - Universidade Federal de Santa Maria (Ufsm)": "CTISM/UFSM",
    "Escola Agrícola de Jundiaí – Universidade Federal do Rio Grande No Norte (Ufrn)": "EAJ/UFRN",
    "Escola de Música - Universidade Federal do Pará (Ufpa)": "EMUFPA",
    "Escola de Música - Universidade Federal do Rio Grande No Norte (Ufrn)": "EMUFRN",
    "Escola de Saúde - Universidade Federal do Rio Grande No Norte (Ufrn)": "ESUFRN",
}

print("Configurações inicializadas!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 7.9 MB/s eta 0:00:00
Dependências e bibliotecas instaladas com sucesso!
Importação de bibliotecas concluída.
Autenticação e cliente gspread inicializados.
Configurações inicializadas!


In [2]:
# ==============================================================================
# @title **SEÇÃO 2: FUNÇÕES UTILITÁRIAS DE EXTRAÇÃO E LIMPEZA INICIAL**
# ==============================================================================


# --- Função de Retentativas (Retry) ---
def solicitar_pagina_com_retry(url: str, headers: dict, max_tentativas: int = 5, timeout: int = 30) -> requests.Response:
    """
    Tenta acessar uma URL com mecanismo de retentativa e backoff exponencial.
    Ideal para lidar com ReadTimeout e instabilidades do servidor.
    """
    for tentativa in range(1, max_tentativas + 1):
        try:
            # Tenta realizar a requisição
            response = requests.get(url, headers=headers, timeout=timeout)
            response.raise_for_status() # Verifica erros HTTP (404, 500, etc)
            return response # Sucesso, retorna a resposta imediatamente

        except (ReadTimeout, ConnectionError) as e:
            # Erros de conexão ou timeout: tenta novamente
            if tentativa < max_tentativas:
                tempo_espera = 2 ** tentativa # Backoff exponencial: 2s, 4s, 8s, 16s...
                logging.warning(f"  -> Tentativa {tentativa}/{max_tentativas} falhou (Timeout/Conexão). Aguardando {tempo_espera}s para tentar novamente...")
                time.sleep(tempo_espera)
            else:
                logging.error(f"  -> Todas as {max_tentativas} tentativas falharam para a URL.")
                raise e # Relança o erro na última tentativa para ser tratado no loop principal

        except HTTPError as e:
            # Erros 4xx ou 5xx: Geralmente não adianta tentar de novo imediatamente (salvo 500/503)
            logging.error(f"  -> Erro HTTP {e.response.status_code} na tentativa {tentativa}.")
            raise e

    return None


def get_uf_sigla(uf_slug: str) -> str:
    """
    Converte o slug de uma UF (usado na URL) para a sigla correspondente.
    Utiliza um dicionário de mapeamento e, como fallback, retorna as duas
    primeiras letras do slug em maiúsculas.

    Args:
        uf_slug: O slug da UF (ex: "sao-paulo").

    Returns:
        A sigla da UF (ex: "SP").
    """
    return SLUG_TO_UF.get(uf_slug, uf_slug.upper()[:2])

# Configuração do logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# --- FUNÇÕES DE NORMALIZAÇÃO E LIMPEZA (usadas por extract_all_fields) ---
def normalize_text(text: Optional[str]) -> Optional[str]:
    """
    Limpa e padroniza uma string, remove acentos, espaços extras e quebras de linha.

    Args:
        text: A string de entrada.

    Returns:
        A string limpa ou None se a entrada for nula.
    """
    if not text:
        return None
    # Substitui múltiplos espaços/quebras de linha por um único espaço e remove
    # espaços no início e no fim.
    return re.sub(r'\s+', ' ', text).strip()

def extract_dirigente(text_block: str, is_reitoria: bool = False) -> Optional[str]:
    """
    Extrai o nome do Reitor(a) ou Diretor(a) do bloco de texto da Reitoria.
    """
    if not text_block:
        return None

    # Padrão regex para "Reitor(a): [Nome]" ou "Diretor(a)-Geral: [Nome]"
    # Captura nomes mesmo com "Pro Tempore"
    patterns = [
        r'(Reitor(a)?( Pro Tempore)?)\s*:\s*([^(\n<]+)',
        r'(Diretor(a)?(-Geral)?( Pro Tempore)?)\s*:\s*([^(\n<]+)'
    ]

    nome_dirigente = None
    for pattern in patterns:
        match = re.search(pattern, text_block, re.IGNORECASE)
        if match:
            nome_dirigente = match.group(4).strip()
            break

    if nome_dirigente:
        # Limpa o nome de lixo textual
        # Remove "Mandato:", "E-mail:", etc. que podem ter sido capturados
        nome_limpo = re.sub(r'\s*(Mandato|E-mail|Telefone|Endereço|Site|CEP).*', '', nome_dirigente, flags=re.IGNORECASE)
        nome_limpo = nome_limpo.strip(" :-.,")

        # Se o nome for muito curto ou longo, provavelmente é um erro
        if 3 < len(nome_limpo) < 70:
            return normalize_text(nome_limpo)

    # --- Fallback Aprimorado (SÓ para reitorias) ---
    # Se a regex falhar e for uma reitoria, tenta pegar a primeira linha
    if is_reitoria:
        lines = [line.strip() for line in text_block.split('\n') if line.strip()]
        if lines:
            first_line = lines[0]
            # Evita pegar "Endereço:", "Telefone:", etc.
            if not re.search(r'^(Endereço|Telefone|E-mail|Site|CEP|Reitoria)', first_line, re.I):
                # Limpa marcadores comuns que não são nomes
                if 'E-mail:' not in first_line and 'Telefone:' not in first_line:
                     # Remove "Mandato:" se estiver na mesma linha
                    nome_limpo = re.sub(r'\s*Mandato.*', '', first_line, flags=re.I).strip(" :-")
                    # Heurística de validação de nome
                    if len(nome_limpo) > 3 and len(nome_limpo) < 70:
                        return normalize_text(nome_limpo)

    return None # Não retorna nada para campi ou se o fallback falhar


def extract_field_from_text(text: str, label: str) -> Optional[str]:
    """
    Extrai o valor de um campo a partir de um bloco de texto, buscando por um rótulo.
    Utiliza regex para encontrar o texto que se segue ao rótulo.
    Aceita variações como 'Telefone(s):', 'E-mails:', 'E-mail(s):'

    Args:
        text: O bloco de texto onde a busca será realizada.
        label: O rótulo do campo a ser extraído (ex: "Telefone", "E-mail").

    Returns:
        O valor extraído e limpo, ou None se não for encontrado.
    """
    if not text:
        return None
    # label pode vir no plural, ex.: label='Telefone' casa com 'Telefone:', 'Telefones:', 'Telefone(s):'
    # Regex para encontrar o rótulo (case-insensitive) seguido por ':' e capturar o texto até o final da linha.
    # pattern = re.compile(f"{label}:\s*(.*)", re.IGNORECASE)
    pattern = re.compile(rf"{label}s?\(?s?\)?:\s*(.*)", re.IGNORECASE)
    match = pattern.search(text)
    if match:
        return normalize_text(match.group(1))
    return None


def parse_full_address(address_text: Optional[str], uf_fallback: str) -> Dict[str, Optional[str]]:
    """
    Analisa uma string de endereço completa e a divide em componentes estruturados.
    A função usa regex para extrair o Município e a UF a partir do endereço completo.

    Args:
        address_text: A string de endereço completa.
        uf_fallback: A sigla da UF a ser usada caso não seja possível extraí-la do texto.

    Returns:
        Um dicionário com os campos 'Endereco_Completo', 'CEP', 'Municipio' e 'UF'.
    """
    if not address_text:
        return {
            'Endereco_Completo': None, 'CEP': None, 'Municipio': None, 'UF': uf_fallback
        }

    address_text = normalize_text(address_text)

    # 1. Extrair CEP
    cep_match = re.search(r'(\d{2}\.?\d{3}-?\d{3})', address_text)
    cep = cep_match.group(1).replace('.', '').replace('-', '') if cep_match else None

    # ==============================================================================
    # Lógica para extrair nomes dos Municípios e respectivas UFs
    # ==============================================================================
    municipio = None
    uf = uf_fallback # Mantém o fallback caso o regex falhe

    # O regex busca pelo padrão (qualquer texto que é o município) [separador] (UF) [texto antes de] CEP
    # Captura formatos como "Cidade, UF. CEP:", "Cidade/UF - CEP:", "Cidade, UF CEP:", "Cidade, UF - CEP" etc.
    # Adicionado '?' ao final de ':', tornando-o opcional para cobrir o padrão ' - CEP'
    pattern = re.compile(r'([\w\s\'.()-]+?)\s*[,/-]\s*([A-Z]{2})[.\s–-]*CEP:?', re.IGNORECASE)
    match = pattern.search(address_text)

    if match:
        # Se o padrão for encontrado, extrai os grupos de captura
        municipio_bruto = match.group(1)
        uf = match.group(2).upper()

        # Limpeza final do nome do município para remover resquícios do bairro.
        # A heurística é que o nome do município é a última parte do texto capturado,
        # geralmente após um ponto final ou hífen que separa o nome do bairro.
        # Ex: "Bairro Centro. Pouso Alegre" -> "Pouso Alegre"
        # Ex: "Recanto dos Pássaros. Barreiras" -> "Barreiras"
        partes = re.split(r'[.-]', municipio_bruto)
        municipio = normalize_text(partes[-1])

    else:
        # Se o regex principal falhar, tenta um padrão mais simples como fallback.
        mun_uf_match = re.search(r'([\w\s\'.]+?),\s*([A-Z]{2})', address_text)
        if mun_uf_match:
            # Aplica a mesma lógica de limpeza para o fallback
            municipio_bruto = mun_uf_match.group(1)
            partes = re.split(r'[.-]', municipio_bruto)
            municipio = normalize_text(partes[-1])
            uf = mun_uf_match.group(2)

    return {
        'Endereco_Completo': address_text,
        'CEP': cep,
        'Municipio': municipio,
        'UF': uf
    }


# ==============================================================================
# PARSER PRINCIPAL COM ESTADO
# ==============================================================================

def parse_state_page(html_content: str, uf_slug: str, source_url: str) -> List:
    """
    Analisa o conteúdo HTML da página de uma UF e extrai os dados de todas as unidades.
    Implementa uma lógica com estado para lidar com múltiplas instituições na mesma página.
    Extrair dados contidos em tags <table> (mesmo que não sejam irmãos imediatos) ou em parágrafos subsequentes.
    Usar a URL da página como o valor da coluna 'Fonte'.

    Args:
        html_content: O conteúdo HTML da página.
        uf_slug: O nome da UF usado na URL (ex: "sao-paulo").
        source_url: A URL completa da página que está sendo analisada.

    Returns:
        Uma lista de dicionários, cada um representando uma unidade da RFEPT.
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    content_area = soup.find(id='content-core')

    if not content_area:
        logging.warning(f"Área de conteúdo 'id=content-core' não encontrada para a UF {uf_slug.upper()}.")
        return [] # Retorna uma lista vazia em caso de erro

    records = [] # Inicializa a lista vazia
    current_institute_info = {}

    # Aplicar find_all com uma lista de tags para capturar todos os blocos de conteúdo relevantes
    content_blocks = content_area.find_all(['p', 'table'])

    for i, block in enumerate(content_blocks):
        # NOVO: quando o bloco for uma <table> com campi (e já temos uma instituição corrente)
        if block.name == 'table' and current_institute_info:
            for cell in block.find_all('td'):
                cell_text = normalize_text(cell.get_text(separator=' ', strip=True))
                if not cell_text:
                    continue

                # Nome do campus (preferindo <b>/<strong>; fallback: texto antes de 'Endereço:')
                header = cell.find(['b', 'strong'])
                campus_name = normalize_text(header.get_text(strip=True)) if header else None
                if not campus_name:
                    parts = re.split(r'Endereç?o?\s*:', cell_text, flags=re.IGNORECASE)  # cobre 'Endereço' e 'Endereco'
                    if parts:
                        campus_name = normalize_text(parts[0])

                email = extract_field_from_text(cell_text, 'E-mail')
                telefone = extract_field_from_text(cell_text, 'Telefone')
                address_str = extract_field_from_text(cell_text, 'Endereço') or extract_field_from_text(cell_text, 'Endereco')
                uf_sigla = get_uf_sigla(uf_slug)
                address_info = parse_full_address(address_str, uf_sigla)

                records.append({
                    'Nome_IF': current_institute_info.get('Nome_IF'),
                    'Campus_IF': campus_name,
                    'Endereco_Completo': address_info['Endereco_Completo'],
                    'CEP': address_info['CEP'],
                    'Municipio': address_info['Municipio'],
                    'UF': address_info['UF'],
                    'Telefone': telefone,
                    'E-mail': email,
                    'Site': current_institute_info.get('Site'),
                    'Dirigente': None,
                    'Fonte': source_url,
                })

            continue

        # Ignora se não for um parágrafo, pois o início de um bloco é sempre um <p>
        if block.name != 'p':
            continue

        text_content = normalize_text(block.get_text(separator=' ', strip=True))
        if not text_content:
            continue

        # is_institute_header = (block.find('strong') or block.find('b')) and text_content.isupper() and len(text_content.split()) > 3

        inst_keywords = (
            "INSTITUTO FEDERAL SUL-RIO-GRANDENSE", # Escrito na página sem 'DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA'
            "INSTITUTO FEDERAL CATARINENSE", # Escrito na página sem 'DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA'
            "INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO SUL DE MINAS GERAIS",
            "CEFET-MG - CENTRO FEDERAL DE EDUCAÇÃO TECNOLÓGICA DE MINAS GERAIS",
            "INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE MINAS GERAIS",
            "INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO NORTE DE MINAS GERAIS",
            "INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA SUDESTE DE MINAS GERAIS", # Escrita errada na página, faltou "DO SUDESTE"
            "INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO TRIÂNGULO MINEIRO",
            "INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO SUL DE MINAS GERAIS",
            "INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA FLUMINENSE",
            "INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO RIO DE JANEIRO",
            "CENTRO FEDERAL DE EDUCAÇÃO TECNOLÓGICA CELSO SUCKOW DA FONSECA",
            "INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DE PERNAMBUCO",
            "INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO SERTÃO PERNAMBUCANO",
            "INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO PIAUÍ",
            "INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA",
            "COLÉGIO PEDRO II",
            "CENTRO TÉCNICO",
            "CENTRO DE EDUCAÇÃO PROFISSIONAL",
            "CENTRO DE ENSINO",
            "INSTITUTO TECNOLÓGICO",
            "COLÉGIO AGRÍCOLA",
            "COLÉGIO TÉCNICO",
            "COLÉGIO POLITÉCNICO",
            "COLÉGIO UNIVERSITÁRIO",
            "ESCOLA AGRÍCOLA",
            "ESCOLA DE ENFERMAGEM",
            "ESCOLA DE SAÚDE",
            "ESCOLA DE MÚSICA",
            "UNIVERSIDADE TECNOLÓGICA" # Caso ds UTFPR
        )

        text_upper = normalize_text(block.get_text(separator=' ', strip=True)).upper()
        bold_like = (block.find('strong') or block.find('b'))

        is_institute_header = (
            bold_like and
            any(k in text_upper for k in inst_keywords) and
            len(text_upper.split()) > 2
        )

        # Verifica se o parágrafo atual parece ser um cabeçalho de campus.
        # Critérios: o texto começa com termos comuns de identificação de campus (case-insensitive).
        is_campus_header = text_content.upper().startswith(CAMPUS_PREFIXES)


        if is_institute_header and not is_campus_header:
            # =========================================================
            # --- Bloco de uma nova Instituição (Reitoria/Sede) ---
            # =========================================================
            logging.info(f"Nova instituição encontrada: {text_content}")

            # Coleta o texto do parágrafo atual + elementos seguintes (p OU table)
            # até encontrar o próximo cabeçalho de instituição ou campus.
            block_text_list = [text_content]

            for next_block in content_blocks[i+1:]:
                next_text = normalize_text(next_block.get_text(separator=' ', strip=True)) if next_block else ""
                if not next_text:
                    continue

                # Detecta se chegou no início do próximo bloco (instituição ou campus)
                stop = False
                if next_block.name == 'p':
                    next_text_upper = next_text.upper()
                    bold_like_next = (next_block.find('strong') or next_block.find('b'))
                    next_is_inst = bold_like_next and any(k in next_text_upper for k in inst_keywords) and len(next_text_upper.split()) > 2
                    next_is_campus = next_text_upper.startswith(CAMPUS_PREFIXES)
                    stop = next_is_inst or next_is_campus

                if stop:
                    break

                # Agora aceita p ou table como conteúdo do bloco do campus
                block_text_list.append(next_text)

            full_block_text = ' '.join(block_text_list)
            # =========================================================

            site = extract_field_from_text(full_block_text, 'Site')
            dirigente = extract_field_from_text(full_block_text, 'Reitor') or extract_field_from_text(full_block_text, 'Reitora') or extract_field_from_text(full_block_text, 'Diretor-Geral') or extract_field_from_text(full_block_text, 'Diretora-Geral')
            email = extract_field_from_text(full_block_text, 'E-mail')
            telefone = extract_field_from_text(full_block_text, 'Telefone')

            address_str = extract_field_from_text(full_block_text, 'Endereço')
            # uf_sigla = uf_slug.split('-')[-1].upper() if '-' in uf_slug else uf_slug.upper()
            uf_sigla = get_uf_sigla(uf_slug)
            address_info = parse_full_address(address_str, uf_sigla)

            current_institute_info = {
                'Nome_IF': text_content,
                'Site': site
            }

            records.append({
                'Nome_IF': text_content,
                'Campus_IF': 'Reitoria/Sede',
                'Endereco_Completo': address_info['Endereco_Completo'],
                'CEP': address_info['CEP'],
                'Municipio': address_info['Municipio'],
                'UF': address_info['UF'],
                'Telefone': telefone,
                'E-mail': email,
                'Site': site,
                'Dirigente': dirigente,
                'Fonte': source_url # Usar 'source_url' para preencher o campo 'Fonte'
            })

        elif is_campus_header and current_institute_info:
            # =========================================================
            # --- Lógica de extração dos blocos de texto dos Campi ---
            # =========================================================
            header_tag = block.find('b') or block.find('strong')
            # campus_name = normalize_text(header_tag.get_text(strip=True)) if header_tag else text_content.splitlines()
            campus_name = normalize_text(header_tag.get_text(strip=True)) if header_tag else text_content

            logging.info(f"  -> Campus encontrado: {campus_name}")

            # =========================================================
            block_text = [text_content]

            # =========================================================
            # Modificação para aceitar <p> e <table> entre o título e os campus
            # =========================================================
            block_text = [text_content]
            for j in range(i + 1, len(content_blocks)):
                sib = content_blocks[j]
                sib_text = normalize_text(sib.get_text(separator=' ', strip=True)) if sib else ""
                if not sib_text:
                    continue

                # Parar quando achar próximo campus ou próxima instituição
                stop = False
                if sib.name == 'p':
                    sib_up = sib_text.upper()
                    bold_like_sib = (sib.find('strong') or sib.find('b'))
                    sib_is_campus = sib_up.startswith(CAMPUS_PREFIXES)
                    sib_is_inst = bold_like_sib and any(k in sib_up for k in inst_keywords) and len(sib_up.split()) > 2
                    stop = sib_is_campus or sib_is_inst

                if stop:
                    break

                # Agora aceitamos p ou table como conteúdo do bloco do campus
                block_text.append(sib_text)

            full_block_text = ' '.join(block_text)
            # =========================================================

            email = extract_field_from_text(full_block_text, 'E-mail')
            telefone = extract_field_from_text(full_block_text, 'Telefone')
            address_str = extract_field_from_text(full_block_text, 'Endereço')
            # uf_sigla = uf_slug.split('-')[-1].upper() if '-' in uf_slug else uf_slug.upper()
            uf_sigla = get_uf_sigla(uf_slug)
            address_info = parse_full_address(address_str, uf_sigla)

            records.append({
                'Nome_IF': current_institute_info.get('Nome_IF'),
                'Campus_IF': campus_name,
                'Endereco_Completo': address_info['Endereco_Completo'],
                'CEP': address_info['CEP'],
                'Municipio': address_info['Municipio'],
                'UF': address_info['UF'],
                'Telefone': telefone,
                'E-mail': email,
                'Site': current_institute_info.get('Site'),
                'Dirigente': None,
                'Fonte': source_url # Usar 'source_url' para preencher o campo 'Fonte'
            })

    return records


# ==============================================================================
# ORQUESTRADOR PRINCIPAL DO SCRAPING
# ==============================================================================

def run_scraper() -> Tuple:
    """
    Orquestra todo o processo de scraping para as 27 UFs.
    Gerencia as requisições HTTP, o parsing e a agregação dos dados.

    Returns:
        Uma tupla contendo:
        - df_scraped: DataFrame com todos os dados extraídos.
        - df_log: DataFrame com o log de execução para cada UF.
    """
    all_records = []
    summary_log = []

    # logging.info("Iniciando o processo de extração de dados da Rede Federal.")
    logging.info("Iniciando o processo de extração de dados da Rede Federal, com retentativas...")

    for uf in UFS:
        url = URL_BASE.format(uf=uf)
        logging.info(f"Processando UF: {uf.upper()} | URL: {url}")

        status = 'Falha'
        units_found = 0
        error_details = ''
        tentativas_realizadas = 0 # Contador para o log

        try:
            # Pausa com o servidor
            time.sleep(1)

            # Chama a função de retry em vez de requests.get direto
            # response = requests.get(url, headers=HEADERS, timeout=20)
            response = solicitar_pagina_com_retry(url, headers=HEADERS, max_tentativas=5, timeout=30)
            response.raise_for_status()  # Lança exceção para status de erro (4xx, 5xx)

            # Passar a variável 'url' como um novo argumento para a função
            uf_records = parse_state_page(response.text, uf, url)

            if uf_records:
                all_records.extend(uf_records)
                units_found = len(uf_records)
                status = 'Sucesso'
                logging.info(f"  -> {units_found} unidades encontradas para {uf.upper()}.")
            else:
                status = 'Sucesso (sem dados)'
                error_details = 'Página acessada, mas nenhum dado estruturado foi encontrado.'
                logging.warning(f"  -> Nenhuma unidade encontrada para {uf.upper()}.")

        # except requests.exceptions.RequestException as e:
        #     error_details = f"Erro de requisição: {type(e).__name__}"
        #     logging.error(f"  -> ERRO ao acessar a URL para {uf.upper()}. Motivo: {e}")

        except RequestException as e:
            # Captura o erro final após todas as tentativas esgotadas
            error_details = f"Falha após múltiplas tentativas: {type(e).__name__}"
            logging.error(f"  -> ERRO FINAL ao processar {uf.upper()}. Motivo: {e}")

        except Exception as e:
            error_details = f"Erro inesperado no parsing: {type(e).__name__}"
            logging.error(f"  -> ERRO inesperado ao processar {uf.upper()}. Motivo: {e}")

        summary_log.append({
            'UF': uf.upper(),
            'Status': status,
            'Unidades Encontradas': units_found,
            'Detalhes do Erro': error_details
        })

    logging.info("Processo de extração finalizado.")

    df_scraped = pd.DataFrame(all_records)
    df_log = pd.DataFrame(summary_log)

    return df_scraped, df_log


# --- Execução Principal ---
if __name__ == '__main__':
    # Esta verificação permite que o script seja importado como um módulo
    # sem executar o scraping automaticamente.

    df_web_scraping_RFEPT, df_execution_log = run_scraper()

    # Exibição dos resultados
    print("\n" + "="*80)
    print("RESUMO DA EXECUÇÃO DO SCRAPING")
    print("="*80)
    print(df_execution_log.to_string())

    if not df_web_scraping_RFEPT.empty:
        print("\n" + "="*80)
        print("AMOSTRA DOS DADOS EXTRAÍDOS (df_web_scraping_RFEPT)")
        print("="*80)
        print(df_web_scraping_RFEPT.info())
        display(df_web_scraping_RFEPT.head(10))

        # --- Etapa Final: Limpeza e Normalização do DataFrame ---
        logging.info("Iniciando limpeza e normalização do DataFrame final.")

        # Remove a sequência "DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA" do nome das instituições
        df_web_scraping_RFEPT['Nome_IF'] = df_web_scraping_RFEPT['Nome_IF'].str.replace(' DE EDUCAÇÃO, CIÊNCIA E TECNOLOGIA', '', regex=False)

        # Substitui 'Reitoria/Sede' por 'Reitoria do {Nome_IF}' na coluna Campus_IF
        df_web_scraping_RFEPT['Campus_IF'] = df_web_scraping_RFEPT.apply(
            lambda row: f"Reitoria do {row['Nome_IF']}" if row['Campus_IF'] == 'Reitoria/Sede' else row['Campus_IF'],
            axis=1
        )

        # Normaliza nomes de campus (capitaliza) - Aplicado após a substituição da Reitoria
        df_web_scraping_RFEPT[['Nome_IF', 'Campus_IF']] = df_web_scraping_RFEPT[['Nome_IF', 'Campus_IF']].astype(str).apply(lambda x: x.str.strip().str.title())

        # Ajusta conectivos para minúsculas após a capitalização
        # Cria uma função auxiliar para aplicar a correção
        def lowercase_connectives(text):
            if isinstance(text, str):
                words = text.split()
                # Mantém 'das' e 'dos' em minúsculo também
                # Adiciona exceções para 'I', 'II', 'III', 'CEFET-MG', 'CEFET-RJ'
                corrected_words = [
                    word.lower() if word.lower() in ['e', 'em', 'da', 'das', 'de', 'do', 'dos'] else word.upper() if word.upper() in ['I', 'II', 'II)', 'III', 'IX', 'CEFET-MG', 'CEFET-RJ', 'UFMT', 'UFMG',
                                                                                                                                'UFV', 'UFRRJ', 'UFRPE', 'UFPA', 'UFPB', 'UFPI', 'UFRN', 'UFSM', 'EMUFPA', 'EMUFRN', 'ESUFRN'] else word
                    for word in words
                ]
                return ' '.join(corrected_words)
            return text # Retorna o valor original se não for string

        df_web_scraping_RFEPT['Nome_IF'] = df_web_scraping_RFEPT['Nome_IF'].apply(lowercase_connectives)
        df_web_scraping_RFEPT['Campus_IF'] = df_web_scraping_RFEPT['Campus_IF'].apply(lowercase_connectives)

        # Limpa a coluna 'Endereco_Completo' removendo informações de telefone e o que segue
        # Expressão regular mais abrangente para cobrir variações
        df_web_scraping_RFEPT['Endereco_Completo'] = df_web_scraping_RFEPT['Endereco_Completo'].str.split(r'\.\s*Telefone\s*:| Telefone\s*: | Telefone \(|\.\s*Telefone \(|\.\s*\}\s*Telefone:\s*\(').str[0].str.strip()


        # --- Padronização da coluna 'CEP' para o formato XXXXX-XXX ---
        # Remove caracteres não numéricos e formata
        df_web_scraping_RFEPT['CEP'] = df_web_scraping_RFEPT['CEP'].astype(str).str.replace(r'\D', '', regex=True)
        # Aplica a formatação se o CEP tiver 8 dígitos
        df_web_scraping_RFEPT['CEP'] = df_web_scraping_RFEPT['CEP'].apply(lambda x: f"{x[:5]}-{x[5:]}" if len(x) == 8 else (x if x != 'nan' else None))


        # Preenche a coluna Região para as novas unidades
        df_web_scraping_RFEPT['Regioes'] = df_web_scraping_RFEPT['UF'].map(MAPA_REGIOES)

        # Reordena as colunas para um layout mais lógico
        col_order = [
            'Nome_IF', 'Campus_IF', 'Regioes', 'UF', 'Municipio',
            'Endereco_Completo', 'CEP', 'Telefone', 'E-mail', 'Site',
            'Dirigente', 'Fonte'
        ]
        df_web_scraping_RFEPT = df_web_scraping_RFEPT.reindex(columns=col_order)

        print("\n" + "="*80)
        print("AMOSTRA DOS DADOS APÓS LIMPEZA E NORMALIZAÇÃO")
        print("="*80)
        display(df_web_scraping_RFEPT.head(10))
    else:
        logging.warning("Nenhum dado foi extraído. O DataFrame final está vazio.")



RESUMO DA EXECUÇÃO DO SCRAPING
                     UF   Status  Unidades Encontradas Detalhes do Erro
0          MINAS-GERAIS  Sucesso                   110                 
1        RIO-DE-JANEIRO  Sucesso                   111                 
2            PERNAMBUCO  Sucesso                    25                 
3                  ACRE  Sucesso                    13                 
4               ALAGOAS  Sucesso                    37                 
5                 AMAPA  Sucesso                    13                 
6              AMAZONAS  Sucesso                    37                 
7                 BAHIA  Sucesso                    82                 
8                 CEARA  Sucesso                    71                 
9      DISTRITO-FEDERAL  Sucesso                    23                 
10       ESPIRITO-SANTO  Sucesso                    46                 
11                GOIAS  Sucesso                    60                 
12             MARANHAO  Sucesso

,Nome_IF,Campus_IF,Endereco_Completo,CEP,Municipio,UF,Telefone,E-mail,Site,Dirigente,Fonte
0,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",Reitoria/Sede,"Avenida Vicente Simões, Nova Pouso Alegre. Pou...",37550000,Pouso Alegre,MG,(35) 3449-6150 E-mail: reitoria@ifsuldeminas.e...,reitoria@ifsuldeminas.edu.br Site: https://por...,https://portal.ifsuldeminas.edu.br/index.php R...,Cleber Ávila Barbosa Mandato: 1º (15/08/2022 a...,https://www.gov.br/mec/pt-br/assuntos/ept/rede...
1,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",None,"Avenida Vicente Simões, Nova Pouso Alegre. Pou...",37550000,Pouso Alegre,MG,(35) 3449-6150 E-mail: reitoria@ifsuldeminas.e...,reitoria@ifsuldeminas.edu.br Site: https://por...,https://portal.ifsuldeminas.edu.br/index.php R...,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...
2,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",Reitor: Cleber Ávila Barbosa Mandato: 1º (15/0...,None,None,None,MG,None,None,https://portal.ifsuldeminas.edu.br/index.php R...,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...
3,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",CAMPUS AVANÇADO CARMO DE MINAS,"Alameda Murilo Eugênio Rubião, s/nº - Bairro C...",37472000,Bairro Chacrinha Carmo de Minas,MG,(35) 99961-4276 E-mail: secretaria.carmodemina...,secretaria.carmodeminas@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php R...,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...
4,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",CAMPUS AVANÇADO TRÊS CORAÇÕES,"Rua Coronel Edgar Cavalcanti de Albuquerque, n...",37410000,Três Corações,MG,(35) 3239-9494 E-mail: gabinete.trescoracoes@i...,gabinete.trescoracoes@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php R...,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...
5,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",CAMPUS INCONFIDENTES,"Praça Tiradentes, 416, Centro. Inconfidentes, ...",37576000,Inconfidentes,MG,(35) 3464-1200 E-mail: gabinete.inconfidentes@...,gabinete.inconfidentes@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php R...,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...
6,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",CAMPUS MACHADO,"Km 03 Rodovia Machado-Paraguaçu, Bairro Santo ...",37750000,Machado,MG,(35) 3295-9700 E-mail: comunica.machado@ifsuld...,comunica.machado@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php R...,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...
7,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",CAMPUS MUZAMBINHO,"Estrada de Muzambinho, Km 35 - Bairro Morro Pr...",37890000,000 Muzambinho,MG,(35) 3571-5051 E-mail: gabinete@muz.ifsuldemin...,gabinete@muz.ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php R...,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...
8,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",CAMPUS PASSOS,"Rua da Penha, 290 - Bairro Penha II Passos/MG ...",37903070,Bairro Penha II Passos,MG,(35) 3526-4856 E-mail: gabinete.passos@ifsulde...,gabinete.passos@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php R...,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...
9,"INSTITUTO FEDERAL DE EDUCAÇÃO, CIÊNCIA E TECNO...",CAMPUS POÇOS DE CALDAS,"Avenida Dirce Pereira Rosa, nº 300, Bairro Jar...",37713100,Poços de Caldas,MG,(35) 3697-4950 E-mail: gabinete.pocos@ifsuldem...,gabinete.pocos@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php R...,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...



AMOSTRA DOS DADOS APÓS LIMPEZA E NORMALIZAÇÃO


,Nome_IF,Campus_IF,Regioes,UF,Municipio,Endereco_Completo,CEP,Telefone,E-mail,Site,Dirigente,Fonte
0,Instituto Federal do Sul de Minas Gerais,Reitoria do Instituto Federal do Sul de Minas ...,Sudeste,MG,Pouso Alegre,"Avenida Vicente Simões, Nova Pouso Alegre. Pou...",37550-000,(35) 3449-6150 E-mail: reitoria@ifsuldeminas.e...,reitoria@ifsuldeminas.edu.br Site: https://por...,https://portal.ifsuldeminas.edu.br/index.php R...,Cleber Ávila Barbosa Mandato: 1º (15/08/2022 a...,https://www.gov.br/mec/pt-br/assuntos/ept/rede...
1,Instituto Federal do Sul de Minas Gerais,None,Sudeste,MG,Pouso Alegre,"Avenida Vicente Simões, Nova Pouso Alegre. Pou...",37550-000,(35) 3449-6150 E-mail: reitoria@ifsuldeminas.e...,reitoria@ifsuldeminas.edu.br Site: https://por...,https://portal.ifsuldeminas.edu.br/index.php R...,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...
2,Instituto Federal do Sul de Minas Gerais,Reitor: Cleber Ávila Barbosa Mandato: 1º (15/0...,Sudeste,MG,None,None,,None,None,https://portal.ifsuldeminas.edu.br/index.php R...,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...
3,Instituto Federal do Sul de Minas Gerais,Campus Avançado Carmo de Minas,Sudeste,MG,Bairro Chacrinha Carmo de Minas,"Alameda Murilo Eugênio Rubião, s/nº - Bairro C...",37472-000,(35) 99961-4276 E-mail: secretaria.carmodemina...,secretaria.carmodeminas@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php R...,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...
4,Instituto Federal do Sul de Minas Gerais,Campus Avançado Três Corações,Sudeste,MG,Três Corações,"Rua Coronel Edgar Cavalcanti de Albuquerque, n...",37410-000,(35) 3239-9494 E-mail: gabinete.trescoracoes@i...,gabinete.trescoracoes@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php R...,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...
5,Instituto Federal do Sul de Minas Gerais,Campus Inconfidentes,Sudeste,MG,Inconfidentes,"Praça Tiradentes, 416, Centro. Inconfidentes, ...",37576-000,(35) 3464-1200 E-mail: gabinete.inconfidentes@...,gabinete.inconfidentes@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php R...,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...
6,Instituto Federal do Sul de Minas Gerais,Campus Machado,Sudeste,MG,Machado,"Km 03 Rodovia Machado-Paraguaçu, Bairro Santo ...",37750-000,(35) 3295-9700 E-mail: comunica.machado@ifsuld...,comunica.machado@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php R...,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...
7,Instituto Federal do Sul de Minas Gerais,Campus Muzambinho,Sudeste,MG,000 Muzambinho,"Estrada de Muzambinho, Km 35 - Bairro Morro Pr...",37890-000,(35) 3571-5051 E-mail: gabinete@muz.ifsuldemin...,gabinete@muz.ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php R...,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...
8,Instituto Federal do Sul de Minas Gerais,Campus Passos,Sudeste,MG,Bairro Penha II Passos,"Rua da Penha, 290 - Bairro Penha II Passos/MG ...",37903-070,(35) 3526-4856 E-mail: gabinete.passos@ifsulde...,gabinete.passos@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php R...,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...
9,Instituto Federal do Sul de Minas Gerais,Campus Poços de Caldas,Sudeste,MG,Poços de Caldas,"Avenida Dirce Pereira Rosa, nº 300, Bairro Jar...",37713-100,(35) 3697-4950 E-mail: gabinete.pocos@ifsuldem...,gabinete.pocos@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php R...,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...


In [3]:
# ==============================================================================
# @title **SEÇÃO 3: RENOMEAÇÃO, PADRONIZAÇÃO E LIMPEZA CUIDADOSA DE COLUNAS**
# ==============================================================================
# Esta seção combina renomeção, padronização da coluna 'Endereco_Completo' e
# limpeza das colunas 'Telefone', 'E-mail', 'Site' e 'Dirigente'
# ==============================================================================

def rename_nome_if(df: pd.DataFrame, renaming_map: dict) -> pd.DataFrame:
    """
    Renomeia os valores na coluna 'Nome_IF' do DataFrame com base em um mapa fornecido.

    Args:
        df: O DataFrame de entrada.
        renaming_map: Um dicionário onde a chave é o nome atual e o valor é o novo nome.

    Returns:
        O DataFrame com a coluna 'Nome_IF' atualizada.
    """
    logging.info("Aplicando renomeação na coluna 'Nome_IF'...")

    if df.empty or not renaming_map:
        return df

    df['Nome_IF'] = df['Nome_IF'].replace(renaming_map)
    logging.info("Renomeação dos IFs concluída.")
    return df


def rename_campus_if(df: pd.DataFrame, mapping: dict) -> pd.DataFrame:
    """
    Renomeia os valores na coluna 'Campus_IF' do DataFrame com base em um mapa fornecido.

    Args:
        df: O DataFrame de entrada.
        mapping: Um dicionário onde a chave é o valor atual e o valor é o novo valor.

    Returns:
        O DataFrame com a coluna 'Campus_IF' atualizada.
    """
    logging.info("Aplicando renomeação na coluna 'Campus_IF'...")

    # Se o DataFrame estiver vazio ou mapping for None, retorna o original
    if df.empty or not mapping:
        return df

    # O parâmetro regex=True é essencial para substituir substrings
    df['Campus_IF'] = df['Campus_IF'].replace(mapping, regex=True)

    logging.info("Renomeação de campus concluída.")
    return df


# --- Função de Padronização de Endereço ---
def padronizar_endereco_para_mapa(row) -> Optional[str]:
    """
    Cria uma string de endereço padronizada e otimizada para APIs de geocodificação
    e ferramentas de visualização como o Looker Studio.

    Esta versão é mais robusta:
    1. Tenta isolar o logradouro/bairro buscando pelo início do bloco de cidade/UF/CEP.
    2. Usa fallbacks para CEP ou apenas Município se o padrão completo falhar.
    3. Retorna o endereço original se dados essenciais estiverem faltando.

    Args:
        row: Uma linha do DataFrame (requer 'Endereco_Completo', 'Municipio', 'UF', 'CEP').

    Returns:
        Uma string de endereço padronizada ou o endereço original em caso de falha.
    """
    try:
        endereco_bruto = row['Endereco_Completo']
        municipio = row['Municipio']
        uf = row['UF']
        cep = row['CEP']

        # Se faltar o endereço base ou o município/UF (chave da extração), retorna o original.
        if pd.isna(endereco_bruto) or not all([municipio, uf]):
            return endereco_bruto

        endereco_limpo = normalize_text(str(endereco_bruto))
        logradouro_bairro = endereco_limpo # Valor padrão

        # --- Estratégia para isolar o logradouro ---
        # O logradouro é o texto ANTES do bloco [Município, UF, CEP].
        # Vamos tentar encontrar o início desse bloco para "fatiar" a string.

        municipio_escaped = re.escape(str(municipio))
        uf_escaped = re.escape(str(uf))

        # Padrão 1: Tenta encontrar [separador]Município[separador]UF
        # (ex: ". Pouso Alegre, MG", " - Caxias/MA")
        padrao_bloco_cidade = rf"[.,–-]\s*{municipio_escaped}\s*[,/-]\s*{uf_escaped}"
        match = re.search(padrao_bloco_cidade, endereco_limpo, re.IGNORECASE)

        if match:
            # Encontrou o padrão. O logradouro é tudo antes dele.
            logradouro_bairro = endereco_limpo[:match.start()].strip(' .,-')
        else:
            # Padrão 2 (Fallback): Não achou o padrão Muncípio/UF. Tenta dividir pelo CEP.
            if cep and isinstance(cep, str) and cep in endereco_limpo:
                logradouro_bairro = endereco_limpo.split(cep)[0].strip(' .,-')
                # Remove um "CEP:" ou "CEP" solto que sobrou no final
                logradouro_bairro = re.sub(r'CEP:?$', '', logradouro_bairro, flags=re.IGNORECASE).strip(' .,-')

            # Padrão 3 (Fallback): Não achou pelo CEP. Tenta dividir pelo Município.
            # (Menos preciso, pode falhar se o nome do município estiver no logradouro)
            elif municipio in endereco_limpo:
                 logradouro_bairro = endereco_limpo.split(municipio)[0].strip(' .,-')

        # Limpeza final: remove "Endereço:" do início, se o scraper tiver pego.
        logradouro_bairro = re.sub(r'^Endereço:\s*', '', logradouro_bairro, flags=re.IGNORECASE).strip()

        # --- Remontagem do Endereço Padronizado ---
        partes_endereco = [
            logradouro_bairro,
            municipio,
            uf
        ]

        # Adiciona o CEP se ele for válido
        if cep and isinstance(cep, str) and cep != 'nan':
            partes_endereco.append(cep)

        # Junta todas as partes com vírgula, removendo quaisquer partes nulas/vazias.
        endereco_final = ', '.join(filter(None, partes_endereco))
        return endereco_final

    except Exception as e:
        logging.error(f"Erro ao padronizar endereço: {e} para a linha {row.name}")
        return row['Endereco_Completo'] # Retorna o original em caso de erro

# --- Funções de Limpeza de Contato (do seu .txt) ---

def clean_telefone(telefone_text: Optional[str]) -> Optional[str]:
    """
    Limpa a string da coluna 'Telefone', removendo texto após marcadores específicos.

    Args:
        telefone_text: A string da coluna 'Telefone'.

    Returns:
        A string limpa ou None se a entrada for nula.
    """
    if not telefone_text:
        return None

    # Remove tudo após ' E-mail: ' (case-insensitive)
    email_marker = re.search(r' E-?mail[ :\s]', telefone_text, re.IGNORECASE)
    if email_marker:
        telefone_text = telefone_text[:email_marker.start()]

    # Remove tudo após ' CAMPUS ' (case-insensitive)
    campus_marker = re.search(r' CAMPUS ', telefone_text, re.IGNORECASE)
    if campus_marker:
        telefone_text = telefone_text[:campus_marker.start()]

    return normalize_text(telefone_text)

def clean_email(email_text: Optional[str]) -> Optional[str]:
    """
    Limpa a string da coluna 'E-mail', removendo texto após marcadores específicos.

    Args:
        email_text: A string da coluna 'E-mail'.

    Returns:
        A string limpa ou None se a entrada for nula.
    """
    if not email_text:
        return None

    # Remove tudo após ' Site: ' (case-insensitive)
    site_marker = re.search(r' Site:', email_text, re.IGNORECASE)
    if site_marker:
        email_text = email_text[:site_marker.start()]

    # Remove tudo após ' CAMPUS ' (case-insensitive)
    campus_marker = re.search(r' CAMPUS ', email_text, re.IGNORECASE)
    if campus_marker:
        email_text = email_text[:campus_marker.start()]

    # Remove tudo após ' ESCOLA ' (case-insensitive)
    escola_marker = re.search(r' ESCOLA ', email_text, re.IGNORECASE)
    if escola_marker:
        email_text = email_text[:escola_marker.start()]

    # Remove tudo após ' COLÉGIO ' (case-insensitive)
    colegio_marker = re.search(r' COLÉGIO ', email_text, re.IGNORECASE)
    if colegio_marker:
        email_text = email_text[:colegio_marker.start()]

    # Remove tudo após ' CENTRO ' (case-insensitive)
    centro_marker = re.search(r' CENTRO ', email_text, re.IGNORECASE)
    if centro_marker:
        email_text = email_text[:centro_marker.start()]

    # Remove tudo após ' UNED ' (case-insensitive)
    uned_marker = re.search(r' UNED ', email_text, re.IGNORECASE)
    if uned_marker:
        email_text = email_text[:uned_marker.start()]

    # Remove tudo após ' INSTITUTO ' (case-insensitive)
    instituto_marker = re.search(r' INSTITUTO ', email_text, re.IGNORECASE)
    if instituto_marker:
        email_text = email_text[:instituto_marker.start()]

    return normalize_text(email_text)


def clean_site(site_text: Optional[str]) -> Optional[str]:
    """
    Limpa a string da coluna 'Site', removendo texto após marcadores específicos.

    Args:
        site_text: A string da coluna 'Site'.

    Returns:
        A string limpa ou None se a entrada for nula.
    """
    if not site_text:
        return None

    # Lista de marcadores para procurar (com regex que ignora espaços extras)
    # Os marcadores são baseados nos seus exemplos:
    # ' Reitor: ', ' Reitora : ', ' Reitora: ', ' Diretora-Geral: ', ' CAMPUS '
    marcadores = [
        r'\sReitor\s*:',
        r'\sReitora\s*:',
        r'\sDiretor-Geral\s*:',
        r'\sDiretora-Geral\s*:',
        r'\sCAMPUS\s',
        r'\sTEATRO\s',
        r'\sCOLÉGIO\s',
        r'\sESCOLA\s',
    ]

    primeira_posicao = len(site_text)
    encontrou = False

    for marcador_regex in marcadores:
        match = re.search(marcador_regex, site_text, re.IGNORECASE)
        if match and match.start() < primeira_posicao:
            primeira_posicao = match.start()
            encontrou = True

    if encontrou:
        site_text = site_text[:primeira_posicao]

    return normalize_text(site_text)


def clean_dirigente(dirigente_text: Optional[str]) -> Optional[str]:
    """
    Limpa a string da coluna 'Dirigente', removendo texto a partir do
    marcador ' Mandato: ' e suas variações.

    Args:
        dirigente_text: A string da coluna 'Dirigente'.

    Returns:
        A string limpa ou None se a entrada for nula.
    """
    if not dirigente_text:
        return None

    # Procura por " Mandato:" (com variações de espaço e case-insensitive)
    # re.IGNORECASE lida com 'Mandato', 'mandato', 'MANDATO'
    # \s* lida com 'Mandato:' ou 'Mandato :'
    mandato_marker = re.search(r'\sMandato\s*:', dirigente_text, re.IGNORECASE)

    if mandato_marker:
        # Se encontrar o marcador, fatia a string antes dele
        dirigente_text = dirigente_text[:mandato_marker.start()]

    # Retorna o texto normalizado (remove espaços extras/quebras de linha)
    return normalize_text(dirigente_text)


# --- Aplicação da Renomeação, Padronização e Limpeza ---
if 'df_web_scraping_RFEPT' in locals() and not df_web_scraping_RFEPT.empty:

    # Aplica a função de renomeação dos IFS
    df_web_scraping_RFEPT = rename_nome_if(df_web_scraping_RFEPT, MAPA_CORRECAO)

    # Aplica a função de renomeação de campus
    df_web_scraping_RFEPT = rename_campus_if(df_web_scraping_RFEPT, MAPA_CORRECAO_CAMPUS)

    # Aplica a padronização à nova coluna 'Endereco_Padronizado'
    logging.info("Iniciando a padronização da coluna de endereços...")
    df_web_scraping_RFEPT['Endereco_Padronizado'] = df_web_scraping_RFEPT.apply(padronizar_endereco_para_mapa, axis=1)

    # Converter a coluna 'Telefone' para string antes de aplicar a função de limpeza
    logging.info("Convertendo a coluna 'Telefone' para string e iniciando a limpeza...")
    df_web_scraping_RFEPT['Telefone'] = df_web_scraping_RFEPT['Telefone'].astype(str).apply(clean_telefone)

    # Converter a coluna 'E-mail' para string antes de aplicar a função de limpeza
    logging.info("Convertendo a coluna 'E-mail' para string e iniciando a limpeza...")
    df_web_scraping_RFEPT['E-mail'] = df_web_scraping_RFEPT['E-mail'].astype(str).apply(clean_email)

    # Converter a coluna 'Site' para string antes de aplicar a função de limpeza
    logging.info("Convertendo a coluna 'Site' para string e iniciando a limpeza...")
    df_web_scraping_RFEPT['Site'] = df_web_scraping_RFEPT['Site'].astype(str).apply(clean_site)

    # Converter a coluna 'Dirigente' para string antes de aplicar a função de limpeza
    logging.info("Convertendo a coluna 'Dirigente' para string e iniciando a limpeza...")
    df_web_scraping_RFEPT['Dirigente'] = df_web_scraping_RFEPT['Dirigente'].astype(str).apply(clean_dirigente)

    # Reordenar o DataFrame com as colunas especificadas pelo usuário
    colunas_ordenadas = [
        'Nome_IF', 'Campus_IF', 'Regioes', 'UF', 'Municipio',
        'Endereco_Completo', 'Endereco_Padronizado', 'CEP', 'Telefone', 'E-mail', 'Site', 'Dirigente', 'Fonte'
    ]

    # Garante que apenas as colunas especificadas e existentes sejam selecionadas
    df_web_scraping_RFEPT = df_web_scraping_RFEPT[colunas_ordenadas].copy()

    print(f"DataFrame possui {df_web_scraping_RFEPT.shape[0]} linhas e {df_web_scraping_RFEPT.shape[1]} colunas após reordenar.")

    print("\n" + "="*80)
    print("EXIBIR DATAFRAME APÓS PADRONIZAÇÃO E LIMPEZA")
    print("="*80)
    # Exibe df_web_scraping_RFEPT após padronização e limpeza
    display(df_web_scraping_RFEPT.head())

else:
    logging.warning("DataFrame 'df_web_scraping_RFEPT' não encontrado.")

DataFrame possui 1269 linhas e 13 colunas após reordenar.

EXIBIR DATAFRAME APÓS PADRONIZAÇÃO E LIMPEZA


,Nome_IF,Campus_IF,Regioes,UF,Municipio,Endereco_Completo,Endereco_Padronizado,CEP,Telefone,E-mail,Site,Dirigente,Fonte
0,Instituto Federal do Sul de Minas Gerais,Reitoria do Instituto Federal do Sul de Minas ...,Sudeste,MG,Pouso Alegre,"Avenida Vicente Simões, Nova Pouso Alegre. Pou...","Avenida Vicente Simões, Nova Pouso Alegre, Pou...",37550-000,(35) 3449-6150,reitoria@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php,Cleber Ávila Barbosa,https://www.gov.br/mec/pt-br/assuntos/ept/rede...
1,Instituto Federal do Sul de Minas Gerais,None,Sudeste,MG,Pouso Alegre,"Avenida Vicente Simões, Nova Pouso Alegre. Pou...","Avenida Vicente Simões, Nova Pouso Alegre, Pou...",37550-000,(35) 3449-6150,reitoria@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...
2,Instituto Federal do Sul de Minas Gerais,Reitor: Cleber Ávila Barbosa Mandato: 1º (15/0...,Sudeste,MG,None,None,None,,None,None,https://portal.ifsuldeminas.edu.br/index.php,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...
3,Instituto Federal do Sul de Minas Gerais,Campus Avançado Carmo de Minas,Sudeste,MG,Bairro Chacrinha Carmo de Minas,"Alameda Murilo Eugênio Rubião, s/nº - Bairro C...","Alameda Murilo Eugênio Rubião, s/nº, Bairro Ch...",37472-000,(35) 99961-4276,secretaria.carmodeminas@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...
4,Instituto Federal do Sul de Minas Gerais,Campus Avançado Três Corações,Sudeste,MG,Três Corações,"Rua Coronel Edgar Cavalcanti de Albuquerque, n...","Rua Coronel Edgar Cavalcanti de Albuquerque, n...",37410-000,(35) 3239-9494,gabinete.trescoracoes@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...


In [4]:
# ==============================================================================
# @title **SEÇÃO 4: ENRIQUECIMENTO - Adição das colunas 'Sigla_IF' e 'UID' e remoção de linhas duplicadas**
# ==============================================================================

# ------------------------------------------------------------------------------
# FUNÇÕES DE APOIO
# ------------------------------------------------------------------------------

def normalize_lookup(text: str) -> str:
    """
    Normaliza o texto para a busca: remove acentos e converte para minúsculas.
    """
    if not isinstance(text, str):
        return ""
    return unidecode(text).lower()

# 1. Cria o dicionário normalizado temporário
temp_map = {
    normalize_lookup(nome): sigla for nome, sigla in MAPA_SIGLAS_IFS.items()
}

# 2. ORDENA o dicionário pelo tamanho da chave (decrescente).
# Isso é CRUCIAL: garante que 'instituto federal do parana' (maior) seja testado
# antes de 'instituto federal do para' (menor), evitando falsos positivos.
MAPA_SIGLAS_NORMALIZADO = dict(sorted(temp_map.items(), key=lambda item: len(item[0]), reverse=True))


def atribuir_sigla(nome_completo_df: str) -> Optional[str]:
    """
    Compara o nome completo do DataFrame com os nomes-base normalizados do mapa.
    Retorna a sigla correspondente se o nome-base estiver contido no nome completo.
    """
    if not isinstance(nome_completo_df, str):
        return np.nan

    nome_normalizado_df = normalize_lookup(nome_completo_df)

    # Itera pelo mapa (agora ordenado do maior para o menor)
    for nome_base_norm, sigla in MAPA_SIGLAS_NORMALIZADO.items():
        if nome_base_norm in nome_normalizado_df:
            return sigla

    return np.nan


def criar_uid_padronizado(texto):
    """
    Recebe uma string, remove acentos, converte para minúsculas
    e substitui caracteres não alfanuméricos por underline.
    """
    if not isinstance(texto, str):
        return str(texto) if pd.notnull(texto) else ""

    # 1. Normalização Unicode (separa o acento da letra)
    texto_normalizado = unicodedata.normalize('NFKD', texto)

    # 2. Mantém apenas caracteres ASCII e converte para minúsculas
    texto_sem_acento = texto_normalizado.encode('ASCII', 'ignore').decode('utf-8').lower()

    # 3. Substitui tudo que NÃO for letra ou número por underline (_)
    texto_limpo = re.sub(r'[^a-z0-9]+', '_', texto_sem_acento)

    # 4. Remove underlines sobrando no início ou fim
    return texto_limpo.strip('_')

def remove_duplicate_rows(df: pd.DataFrame) -> pd.DataFrame:
    """
    Remove linhas duplicadas de um DataFrame com base na coluna 'UID'
    e filtra linhas com valores indesejados na coluna 'Campus_IF',
    mantendo a primeira ocorrência.
    """
    logging.info("Iniciando remoção de linhas duplicadas e filtragem de Campus_IF inválido...")

    if df is None or df.empty:
        logging.warning("DataFrame vazio ou None recebido em remove_duplicate_rows.")
        return df

    original_rows = df.shape[0]

    # Converte 'Campus_IF' para string para aplicar métodos string de forma segura
    campus_if_str = df['Campus_IF'].astype(str).str.lower()

    # Define as condições para filtrar linhas inválidas em 'Campus_IF'
    # Regex ajustado para pegar 'none', 'reitor:', 'diretora-geral:', 'nan' ou apenas um ponto '.'
    invalid_campus_conditions = campus_if_str.str.contains(
        r'none|reitor(?:a)?:|diretor(?:a)?-geral:|^nan$|^\.$',
        regex=True, na=False
    )

    # Filtra o DataFrame, mantendo apenas as linhas que NÃO satisfazem as condições inválidas
    df_filtered = df[~invalid_campus_conditions].copy()

    filtered_out_count = original_rows - df_filtered.shape[0]
    if filtered_out_count > 0:
        logging.info(f" -> {filtered_out_count} linhas filtradas devido a valores inválidos em 'Campus_IF'.")

    # Remove duplicatas com base na coluna 'UID' do DataFrame já filtrado
    df_cleaned = df_filtered.drop_duplicates(subset=['UID'], keep='first')

    removed_duplicates_count = df_filtered.shape[0] - df_cleaned.shape[0]
    if removed_duplicates_count > 0:
        logging.info(f" -> {removed_duplicates_count} linhas duplicadas removidas com base em 'UID'.")

    total_removed = original_rows - df_cleaned.shape[0]
    logging.info(f"Remoção e filtragem concluídas: {total_removed} linhas removidas no total.")

    return df_cleaned

# ------------------------------------------------------------------------------
# APLICAÇÃO E REORDENAÇÃO
# ------------------------------------------------------------------------------

if 'df_web_scraping_RFEPT' in locals() and not df_web_scraping_RFEPT.empty:
    logging.info("Iniciando processo da Seção 4...")

    # 1. Cria a nova coluna 'Sigla_IF'
    logging.info("Criando coluna 'Sigla_IF'...")
    df_web_scraping_RFEPT['Sigla_IF'] = df_web_scraping_RFEPT['Nome_IF'].apply(atribuir_sigla)

    # 2. Cria a coluna 'UID' (Identificador Único)
    logging.info("Criando coluna 'UID'...")
    coluna_bruta = (
        df_web_scraping_RFEPT['Sigla_IF'].astype(str).str.strip() + '_' +
        df_web_scraping_RFEPT['Campus_IF'].astype(str).str.strip()
    )
    df_web_scraping_RFEPT['UID'] = coluna_bruta.apply(criar_uid_padronizado)

    # Imprimir status antes da limpeza
    print(f"\n{'='*80}")
    print(f"Shape do DataFrame ANTES da limpeza: {df_web_scraping_RFEPT.shape}")

    # 3. Aplicação da função remove_duplicate_rows
    df_web_scraping_RFEPT = remove_duplicate_rows(df_web_scraping_RFEPT)
    print(f"Shape do DataFrame APÓS limpeza: {df_web_scraping_RFEPT.shape}")
    print(f"{'='*80}")

    # 4. Reordenação das colunas
    try:
        logging.info("Reordenando colunas...")
        cols = list(df_web_scraping_RFEPT.columns)

        # Move 'Sigla_IF' para antes de 'Nome_IF' se ela não estiver na posição certa
        if 'Sigla_IF' in cols and 'Nome_IF' in cols:
            idx_nome_if = df_web_scraping_RFEPT.columns.get_loc('Nome_IF')
            cols.pop(cols.index('Sigla_IF'))
            cols.insert(idx_nome_if, 'Sigla_IF')

        # Move 'UID' para o início (opcional, mas comum para identificadores)
        if 'UID' in cols:
            cols.pop(cols.index('UID'))
            cols.insert(0, 'UID')

        df_web_scraping_RFEPT = df_web_scraping_RFEPT[cols]

        print("\n" + "="*80)
        print("DATAFRAME APÓS INCLUSÃO DAS NOVAS COLUNAS")
        print("="*80)

        # Filtra colunas para exibição apenas se elas existirem
        cols_to_show = ['UID', 'Sigla_IF', 'Nome_IF', 'Campus_IF', 'UF', 'Municipio', 'Telefone', 'E-mail', 'Dirigente']
        cols_existing = [c for c in cols_to_show if c in df_web_scraping_RFEPT.columns]

        display(df_web_scraping_RFEPT[cols_existing].head())

    except Exception as e:
        logging.error(f"Erro ao reordenar colunas: {e}")
        print(f"Erro não fatal na reordenação: {e}")

else:
    logging.warning("DataFrame 'df_web_scraping_RFEPT' não encontrado.")
    print("DataFrame 'df_web_scraping_RFEPT' não encontrado. Execute as seções anteriores.")


Shape do DataFrame ANTES da limpeza: (1269, 15)
Shape do DataFrame APÓS limpeza: (715, 15)

DATAFRAME APÓS INCLUSÃO DAS NOVAS COLUNAS


,UID,Sigla_IF,Nome_IF,Campus_IF,UF,Municipio,Telefone,E-mail,Dirigente
0,ifsuldeminas_reitoria_do_instituto_federal_do_...,IFSULDEMINAS,Instituto Federal do Sul de Minas Gerais,Reitoria do Instituto Federal do Sul de Minas ...,MG,Pouso Alegre,(35) 3449-6150,reitoria@ifsuldeminas.edu.br,Cleber Ávila Barbosa
3,ifsuldeminas_campus_avancado_carmo_de_minas,IFSULDEMINAS,Instituto Federal do Sul de Minas Gerais,Campus Avançado Carmo de Minas,MG,Bairro Chacrinha Carmo de Minas,(35) 99961-4276,secretaria.carmodeminas@ifsuldeminas.edu.br,None
4,ifsuldeminas_campus_avancado_tres_coracoes,IFSULDEMINAS,Instituto Federal do Sul de Minas Gerais,Campus Avançado Três Corações,MG,Três Corações,(35) 3239-9494,gabinete.trescoracoes@ifsuldeminas.edu.br,None
5,ifsuldeminas_campus_inconfidentes,IFSULDEMINAS,Instituto Federal do Sul de Minas Gerais,Campus Inconfidentes,MG,Inconfidentes,(35) 3464-1200,gabinete.inconfidentes@ifsuldeminas.edu.br,None
6,ifsuldeminas_campus_machado,IFSULDEMINAS,Instituto Federal do Sul de Minas Gerais,Campus Machado,MG,Machado,(35) 3295-9700,comunica.machado@ifsuldeminas.edu.br,None


In [5]:
# ========================================================================================================
# @title **SEÇÃO 5: ENRIQUECIMENTO - Merge de linhas com dataset do MEC `df_mapa_RFEPT`, inclusão do 'Ano_Criacao'**
# ========================================================================================================

# Opcionalmente
# url_csv = 'https://raw.githubusercontent.com/SampMark/files/refs/heads/main/REDE%20FEDERAL%20DE%20EPCT%20%E2%80%93%20Unidades%20da%20Rede%20Federal%20de%20EPCT%20-%20Mapa_RFEPCT_MEC_2022.csv'
# df_mapa_RFEPT = pd.read_csv(url_csv, encoding='utf-8')

# --- Configurações da Planilha Online ---
# ID da planilha
ID_PLANILHA = '18S79OGp2RV5QmxO3Rb39PCmRkY_WrCL3gWgRGqIPELk' # Planilha compartilhada publicamente
NOME_ABA = 'Mapa_RFEPCT_MEC_2022'

print(f"Tentando carregar dados da planilha '{NOME_ABA}' (ID: {ID_PLANILHA})...")

try:
    # Assume que o cliente gspread 'gc' já está autenticado e inicializado.
    # Se não estiver, execute a célula L860O5xSTr1n antes desta.
    sh = gc.open_by_key(ID_PLANILHA)
    worksheet = sh.worksheet(NOME_ABA)

    # Obter todos os registros como lista de dicionários
    data = worksheet.get_all_records()

    # Converter para DataFrame
    df_mapa_RFEPT = pd.DataFrame(data)

    print(f"DataFrame 'df_mapa_RFEPT' carregado com sucesso da aba '{NOME_ABA}'!")
    print(f"O 'df_mapa_RFEPT' possui {df_mapa_RFEPT.shape[0]} linhas e {df_mapa_RFEPT.shape[1]} colunas.")
    print("\n" + "="*80)
    print("AMOSTRA DOS DADOS DO DATAFRAME 'df_mapa_RFEPT'")
    print("="*80)
    display(df_mapa_RFEPT.head())

except gspread.exceptions.SpreadsheetNotFound:
    print(f"ERRO: Planilha com ID '{ID_PLANILHA}' não encontrada. Verifique o ID.")
except gspread.exceptions.WorksheetNotFound:
    print(f"ERRO: Aba '{NOME_ABA}' não encontrada na planilha. Verifique o nome da aba.")
except Exception as e:
    print(f"Ocorreu um erro ao carregar o df_mapa_RFEPT: {e}")
# --- Fim importação de Planilha Online ---

print("\n" + "="*80)
print("COMPARAÇÃO DE LINHAS ENTRE df_web_scraping_RFEPT E df_mapa_RFEPT")
print("COM BASE NA COLUNA 'UID'")
print("="*80)

# Garante que as colunas 'UID' são do tipo string para comparações consistentes
df_web_scraping_RFEPT['UID'] = df_web_scraping_RFEPT['UID'].astype(str)
df_mapa_RFEPT['UID'] = df_mapa_RFEPT['UID'].astype(str)

# 1. Conjuntos de UIDs
uids_web_scraping = set(df_web_scraping_RFEPT['UID'])
uids_mapa = set(df_mapa_RFEPT['UID'])

# 2. Correspondências (UIDs em ambos os DataFrames)
uids_comuns = uids_web_scraping.intersection(uids_mapa)
print(f"\nTotal de UIDs em df_web_scraping_RFEPT: {len(uids_web_scraping)}")
print(f"Total de UIDs em df_mapa_RFEPT: {len(uids_mapa)}")
print(f"Total de UIDs comuns a ambos os DataFrames: {len(uids_comuns)}")

# 3. Diferenças (UIDs únicos para cada DataFrame)
uids_apenas_web_scraping = uids_web_scraping.difference(uids_mapa)
uids_apenas_mapa = uids_mapa.difference(uids_web_scraping)

print(f"UIDs encontrados apenas em df_web_scraping_RFEPT: {len(uids_apenas_web_scraping)}")
print(f"UIDs encontrados apenas em df_mapa_RFEPT: {len(uids_apenas_mapa)}")

# Exibição de amostras para análise qualitativa
print("\n" + "="*80)
print("AMOSTRA DE UIDs COMUNS:")
print("="*80)
if uids_comuns:
    # Seleciona algumas linhas correspondentes de ambos os DFs para visualização
    sample_uids_comuns = list(uids_comuns)[:5]
    print("UIDs comuns:")
    for uid in sample_uids_comuns:
        print(f"- {uid}")

    # Display details from both dataframes for common UIDs
    print("\nDetalhes de amostra para UIDs comuns (df_web_scraping_RFEPT):")
    display(df_web_scraping_RFEPT[df_web_scraping_RFEPT['UID'].isin(sample_uids_comuns)][['UID', 'Nome_IF', 'Campus_IF']].head())
    print("\nDetalhes de amostra para UIDs comuns (df_mapa_RFEPT):")
    display(df_mapa_RFEPT[df_mapa_RFEPT['UID'].isin(sample_uids_comuns)][['UID', 'Nome_IF', 'Campus_IF']].head())
else:
    print("Não há UIDs comuns.")

print("\n" + "="*80)
print("AMOSTRA DE UIDs APENAS EM df_web_scraping_RFEPT:")
print("="*80)
if uids_apenas_web_scraping:
    sample_uids_web = list(uids_apenas_web_scraping)[:5]
    print("UIDs (Web Scraping):")
    for uid in sample_uids_web:
        print(f"- {uid}")
    display(df_web_scraping_RFEPT[df_web_scraping_RFEPT['UID'].isin(sample_uids_web)][['UID', 'Nome_IF', 'Campus_IF']].head())
else:
    print("Não há UIDs exclusivos no df_web_scraping_RFEPT.")

print("\n" + "="*80)
print("AMOSTRA DE UIDs APENAS EM df_mapa_RFEPT:")
print("="*80)
if uids_apenas_mapa:
    sample_uids_mapa = list(uids_apenas_mapa)[:5]
    print("UIDs (Mapa Original):")
    for uid in sample_uids_mapa:
        print(f"- {uid}")
    display(df_mapa_RFEPT[df_mapa_RFEPT['UID'].isin(sample_uids_mapa)][['UID', 'Nome_IF', 'Campus_IF']].head())
else:
    print("Não há UIDs exclusivos no df_mapa_RFEPT.")


Tentando carregar dados da planilha 'Mapa_RFEPCT_MEC_2022' (ID: 18S79OGp2RV5QmxO3Rb39PCmRkY_WrCL3gWgRGqIPELk)...
DataFrame 'df_mapa_RFEPT' carregado com sucesso da aba 'Mapa_RFEPCT_MEC_2022'!
O 'df_mapa_RFEPT' possui 644 linhas e 9 colunas.

AMOSTRA DOS DADOS DO DATAFRAME 'df_mapa_RFEPT'


,UID,Ano_Criacao,Sigla_IF,Nome_IF,Campus_IF,Regioes,UF,Municipio,Endereco_Padronizado
0,ifac_campus_rio_branco,2010,IFAC,Instituto Federal do Acre,Campus Rio Branco,Norte,AC,Rio Branco,Rio Branco/AC
1,ifac_campus_sena_madureira,2010,IFAC,Instituto Federal do Acre,Campus Sena Madureira,Norte,AC,Sena Madureira,Sena Madureira/AC
2,ifac_campus_tarauaca,2013,IFAC,Instituto Federal do Acre,Campus Tarauacá,Norte,AC,Tarauacá,Tarauacá/AC
3,ifac_campus_cruzeiro_do_sul,2010,IFAC,Instituto Federal do Acre,Campus Cruzeiro do Sul,Norte,AC,Cruzeiro do Sul,Cruzeiro do Sul/AC
4,ifac_campus_xapuri,2013,IFAC,Instituto Federal do Acre,Campus Xapuri,Norte,AC,Xapuri,Xapuri/AC



COMPARAÇÃO DE LINHAS ENTRE df_web_scraping_RFEPT E df_mapa_RFEPT
COM BASE NA COLUNA 'UID'

Total de UIDs em df_web_scraping_RFEPT: 715
Total de UIDs em df_mapa_RFEPT: 644
Total de UIDs comuns a ambos os DataFrames: 625
UIDs encontrados apenas em df_web_scraping_RFEPT: 90
UIDs encontrados apenas em df_mapa_RFEPT: 19

AMOSTRA DE UIDs COMUNS:
UIDs comuns:
- ifmt_campus_sao_vicente
- ifsul_campus_pelotas_visconde_da_graca
- ifmg_campus_ipatinga
- ifma_campus_sao_luis_centro_historico
- ifpb_campus_santa_rita

Detalhes de amostra para UIDs comuns (df_web_scraping_RFEPT):


,UID,Nome_IF,Campus_IF
49,ifmg_campus_ipatinga,Instituto Federal de Minas Gerais,Campus Ipatinga
655,ifma_campus_sao_luis_centro_historico,Instituto Federal do Maranhão,Campus São Luís Centro Histórico
706,ifmt_campus_sao_vicente,Instituto Federal de Mato Grosso,Campus São Vicente
793,ifpb_campus_santa_rita,Instituto Federal da Paraíba,Campus Santa Rita
965,ifsul_campus_pelotas_visconde_da_graca,Instituto Federal Sul-Rio-Grandense,Campus Pelotas-Visconde da Graça



Detalhes de amostra para UIDs comuns (df_mapa_RFEPT):


,UID,Nome_IF,Campus_IF
182,ifma_campus_sao_luis_centro_historico,Instituto Federal do Maranhão,Campus São Luís Centro Histórico
228,ifmg_campus_ipatinga,Instituto Federal de Minas Gerais,Campus Ipatinga
291,ifmt_campus_sao_vicente,Instituto Federal de Mato Grosso,Campus São Vicente
339,ifpb_campus_santa_rita,Instituto Federal da Paraíba,Campus Santa Rita
515,ifsul_campus_pelotas_visconde_da_graca,Instituto Federal Sul-rio-grandense,Campus Pelotas Visconde da Graça



AMOSTRA DE UIDs APENAS EM df_web_scraping_RFEPT:
UIDs (Web Scraping):
- ifsc_reitoria_do_instituto_federal_de_santa_catarina
- ifro_reitoria_do_instituto_federal_de_rondonia
- ifes_reitoria_do_instituto_federal_do_espirito_santo
- ifap_campus_pedra_branca_do_amapari
- ifma_campus_vitorino_freire


,UID,Nome_IF,Campus_IF
302,ifap_campus_pedra_branca_do_amapari,Instituto Federal do Amapá,Campus Pedra Branca do Amapari
522,ifes_reitoria_do_instituto_federal_do_espirito...,Instituto Federal do Espírito Santo,Reitoria do Instituto Federal do Espírito Santo
661,ifma_campus_vitorino_freire,Instituto Federal do Maranhão,Campus Vitorino Freire
1043,ifro_reitoria_do_instituto_federal_de_rondonia,Instituto Federal de Rondonia,Reitoria do Instituto Federal de Rondonia
1065,ifsc_reitoria_do_instituto_federal_de_santa_ca...,Instituto Federal de Santa Catarina,Reitoria do Instituto Federal de Santa Catarina



AMOSTRA DE UIDs APENAS EM df_mapa_RFEPT:
UIDs (Mapa Original):
- iftm_polo_de_inovacao_uberaba
- ifpb_polo_de_inovacao_joao_pessoa
- cefet_rj_unidade_maracana
- ifrn_campus_avancado_jucurutu
- if_goiano_polo_de_inovacao_rio_verde


,UID,Nome_IF,Campus_IF
163,if_goiano_polo_de_inovacao_rio_verde,Instituto Federal Goiano,Polo de Inovação Rio Verde
275,iftm_polo_de_inovacao_uberaba,Instituto Federal do Triângulo Mineiro,Polo de Inovação Uberaba
342,ifpb_polo_de_inovacao_joao_pessoa,Instituto Federal da Paraíba,Polo de Inovação João Pessoa
414,cefet_rj_unidade_maracana,Centro Federal de Educação Tecnológica Celso S...,Unidade Maracanã
484,ifrn_campus_avancado_jucurutu,Instituto Federal do Rio Grande do Norte,Campus Avançado Jucurutu


In [6]:
# Obter as colunas de cada DataFrame
colunas_web_scraping = set(df_web_scraping_RFEPT.columns)
colunas_mapa_RFEPT = set(df_mapa_RFEPT.columns)

# Encontrar as colunas comuns
colunas_comuns = colunas_web_scraping.intersection(colunas_mapa_RFEPT)

print("Colunas em df_web_scraping_RFEPT:", sorted(list(colunas_web_scraping)))
print("Colunas em df_mapa_RFEPT:", sorted(list(colunas_mapa_RFEPT)))
print("\nColunas comuns a ambos os DataFrames:", sorted(list(colunas_comuns)))


Colunas em df_web_scraping_RFEPT: ['CEP', 'Campus_IF', 'Dirigente', 'E-mail', 'Endereco_Completo', 'Endereco_Padronizado', 'Fonte', 'Municipio', 'Nome_IF', 'Regioes', 'Sigla_IF', 'Site', 'Telefone', 'UF', 'UID']
Colunas em df_mapa_RFEPT: ['Ano_Criacao', 'Campus_IF', 'Endereco_Padronizado', 'Municipio', 'Nome_IF', 'Regioes', 'Sigla_IF', 'UF', 'UID']

Colunas comuns a ambos os DataFrames: ['Campus_IF', 'Endereco_Padronizado', 'Municipio', 'Nome_IF', 'Regioes', 'Sigla_IF', 'UF', 'UID']


In [7]:
def incluir_ano_criacao(df_web_scraping: pd.DataFrame, df_mapa: pd.DataFrame) -> pd.DataFrame:
    """
    Inclui a coluna 'Ano_Criacao' em df_web_scraping_RFEPT, fazendo um merge
    com df_mapa_RFEPT baseado na coluna 'UID'.

    Args:
        df_web_scraping: DataFrame resultante do web scraping.
        df_mapa: DataFrame carregado do CSV original.

    Returns:
        DataFrame df_web_scraping_RFEPT com a coluna 'Ano_Criacao' adicionada.
    """
    logging.info("Incluindo a coluna 'Ano_Criacao' em df_web_scraping_RFEPT...")

    # Check if 'Ano_Criacao' already exists in df_web_scraping and drop it
    # to prevent suffix renaming during merge.
    if 'Ano_Criacao' in df_web_scraping.columns:
        df_web_scraping = df_web_scraping.drop(columns=['Ano_Criacao'])

    df_merged = pd.merge(
        df_web_scraping,
        df_mapa[['UID', 'Ano_Criacao']],
        on='UID',
        how='left'
    )
    logging.info("Coluna 'Ano_Criacao' incluída.")
    return df_merged


def adicionar_linhas_unicas_mapa(df_web_scraping: pd.DataFrame, df_mapa: pd.DataFrame) -> pd.DataFrame:
    """
    Adiciona ao df_web_scraping_RFEPT as linhas (unidades) encontradas
    apenas no df_mapa_RFEPT, considerando um conjunto de colunas comuns.

    Args:
        df_web_scraping: DataFrame resultante do web scraping.
        df_mapa: DataFrame carregado do CSV original.

    Returns:
        DataFrame df_web_scraping_RFEPT com as linhas únicas de df_mapa_RFEPT adicionadas.
    """
    logging.info("Adicionando linhas únicas de df_mapa_RFEPT a df_web_scraping_RFEPT...")

    # 1. Identificar UIDs únicos em df_mapa que não estão em df_web_scraping
    uids_apenas_mapa = set(df_mapa['UID']).difference(set(df_web_scraping['UID']))
    df_mapa_unicas = df_mapa[df_mapa['UID'].isin(uids_apenas_mapa)].copy()

    if df_mapa_unicas.empty:
        logging.info("Não foram encontradas linhas únicas em df_mapa_RFEPT para adicionar.")
        return df_web_scraping

    # 2. Definir colunas comuns para concatenação
    common_cols_requested = ['UID', 'Ano_Criacao', 'Sigla_IF', 'Nome_IF', 'Campus_IF', 'Regioes', 'UF', 'Municipio', 'Endereco_Padronizado']

    # Filtrar colunas existentes em ambos os DataFrames
    common_cols_web = [col for col in common_cols_requested if col in df_web_scraping.columns]
    common_cols_mapa = [col for col in common_cols_requested if col in df_mapa_unicas.columns]

    # Garantir que a interseção das colunas está presente em ambos antes de selecionar
    final_common_cols = list(set(common_cols_web) & set(common_cols_mapa))

    if not final_common_cols:
        logging.warning("Não há colunas comuns suficientes para concatenar as linhas únicas de df_mapa_RFEPT.")
        return df_web_scraping

    # Selecionar apenas as colunas comuns para as linhas únicas do mapa
    df_mapa_unicas_selected = df_mapa_unicas[final_common_cols]

    # Criar um DataFrame vazio para preencher as colunas que faltam no df_mapa_unicas_selected
    # mas que existem em df_web_scraping, preenchendo-as com None/NaN
    df_fill = pd.DataFrame(columns=[col for col in df_web_scraping.columns if col not in final_common_cols])
    df_mapa_unicas_final = pd.concat([df_mapa_unicas_selected, df_fill], axis=1)

    # Garantir que a ordem das colunas seja a mesma antes de concatenar
    df_mapa_unicas_final = df_mapa_unicas_final[df_web_scraping.columns]

    # 3. Concatenar os DataFrames
    df_final = pd.concat([df_web_scraping, df_mapa_unicas_final], ignore_index=True)
    logging.info(f"{len(uids_apenas_mapa)} linhas únicas de df_mapa_RFEPT adicionadas. Total de linhas: {df_final.shape[0]}")

    return df_final

# --- Aplicação das funções ---

if 'df_web_scraping_RFEPT' in locals() and not df_web_scraping_RFEPT.empty and \
   'df_mapa_RFEPT' in locals() and not df_mapa_RFEPT.empty:

    # 1. Incluir 'Ano_Criacao'
    df_web_scraping_RFEPT = incluir_ano_criacao(df_web_scraping_RFEPT, df_mapa_RFEPT)

    # Converter 'Ano_Criacao' para tipo 'object', tratando NaNs como None ou string vazia
    df_web_scraping_RFEPT['Ano_Criacao'] = df_web_scraping_RFEPT['Ano_Criacao'].fillna('').astype(object)

    # 2. Adicionar linhas únicas do df_mapa_RFEPT
    df_web_scraping_RFEPT = adicionar_linhas_unicas_mapa(df_web_scraping_RFEPT, df_mapa_RFEPT)

    print("\n" + "="*80)
    print("DATAFRAME df_web_scraping_RFEPT APÓS INCLUSÃO DE 'Ano_Criacao' E LINHAS ÚNICAS")
    print("="*80)
    print(f"Novo shape do DataFrame: {df_web_scraping_RFEPT.shape}")
    display(df_web_scraping_RFEPT.head(10))
    display(df_web_scraping_RFEPT.tail(10))

else:
    logging.warning("Um ou ambos os DataFrames (df_web_scraping_RFEPT, df_mapa_RFEPT) não foram encontrados ou estão vazios. Nenhuma operação foi realizada.")


DATAFRAME df_web_scraping_RFEPT APÓS INCLUSÃO DE 'Ano_Criacao' E LINHAS ÚNICAS
Novo shape do DataFrame: (734, 16)


,UID,Sigla_IF,Nome_IF,Campus_IF,Regioes,UF,Municipio,Endereco_Completo,Endereco_Padronizado,CEP,Telefone,E-mail,Site,Dirigente,Fonte,Ano_Criacao
0,ifsuldeminas_reitoria_do_instituto_federal_do_...,IFSULDEMINAS,Instituto Federal do Sul de Minas Gerais,Reitoria do Instituto Federal do Sul de Minas ...,Sudeste,MG,Pouso Alegre,"Avenida Vicente Simões, Nova Pouso Alegre. Pou...","Avenida Vicente Simões, Nova Pouso Alegre, Pou...",37550-000,(35) 3449-6150,reitoria@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php,Cleber Ávila Barbosa,https://www.gov.br/mec/pt-br/assuntos/ept/rede...,
1,ifsuldeminas_campus_avancado_carmo_de_minas,IFSULDEMINAS,Instituto Federal do Sul de Minas Gerais,Campus Avançado Carmo de Minas,Sudeste,MG,Bairro Chacrinha Carmo de Minas,"Alameda Murilo Eugênio Rubião, s/nº - Bairro C...","Alameda Murilo Eugênio Rubião, s/nº, Bairro Ch...",37472-000,(35) 99961-4276,secretaria.carmodeminas@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...,2014.0
2,ifsuldeminas_campus_avancado_tres_coracoes,IFSULDEMINAS,Instituto Federal do Sul de Minas Gerais,Campus Avançado Três Corações,Sudeste,MG,Três Corações,"Rua Coronel Edgar Cavalcanti de Albuquerque, n...","Rua Coronel Edgar Cavalcanti de Albuquerque, n...",37410-000,(35) 3239-9494,gabinete.trescoracoes@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...,2014.0
3,ifsuldeminas_campus_inconfidentes,IFSULDEMINAS,Instituto Federal do Sul de Minas Gerais,Campus Inconfidentes,Sudeste,MG,Inconfidentes,"Praça Tiradentes, 416, Centro. Inconfidentes, ...","Praça Tiradentes, 416, Centro, Inconfidentes, ...",37576-000,(35) 3464-1200,gabinete.inconfidentes@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...,1918.0
4,ifsuldeminas_campus_machado,IFSULDEMINAS,Instituto Federal do Sul de Minas Gerais,Campus Machado,Sudeste,MG,Machado,"Km 03 Rodovia Machado-Paraguaçu, Bairro Santo ...","Km 03 Rodovia Machado-Paraguaçu, Bairro Santo ...",37750-000,(35) 3295-9700,comunica.machado@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...,1946.0
5,ifsuldeminas_campus_muzambinho,IFSULDEMINAS,Instituto Federal do Sul de Minas Gerais,Campus Muzambinho,Sudeste,MG,000 Muzambinho,"Estrada de Muzambinho, Km 35 - Bairro Morro Pr...","Estrada de Muzambinho, Km 35 - Bairro Morro Pr...",37890-000,(35) 3571-5051,gabinete@muz.ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...,1948.0
6,ifsuldeminas_campus_passos,IFSULDEMINAS,Instituto Federal do Sul de Minas Gerais,Campus Passos,Sudeste,MG,Bairro Penha II Passos,"Rua da Penha, 290 - Bairro Penha II Passos/MG ...","Rua da Penha, 290, Bairro Penha II Passos, MG,...",37903-070,(35) 3526-4856,gabinete.passos@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...,2013.0
7,ifsuldeminas_campus_pocos_de_caldas,IFSULDEMINAS,Instituto Federal do Sul de Minas Gerais,Campus Poços de Caldas,Sudeste,MG,Poços de Caldas,"Avenida Dirce Pereira Rosa, nº 300, Bairro Jar...","Avenida Dirce Pereira Rosa, nº 300, Bairro Jar...",37713-100,(35) 3697-4950,gabinete.pocos@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...,2013.0
8,ifsuldeminas_campus_pouso_alegre,IFSULDEMINAS,Instituto Federal do Sul de Minas Gerais,Campus Pouso Alegre,Sudeste,MG,Bairro Parque Real Pouso Alegre,"Avenida Maria da Conceição Santos, nº 900, Bai...","Avenida Maria da Conceição Santos, nº 900, Bai...",37560-260,(35) 3427-6600,gabinete.pousoalegre@ifsuldeminas.edu.br,https://portal.ifsuldeminas.edu.br/index.php,None,https://www.gov.br/mec/pt-br/assuntos/ept/rede...,2013.0
9,cefet_mg_reitoria_do_centro_federal_de_educaca...,CEFET-MG,C

,UID,Sigla_IF,Nome_IF,Campus_IF,Regioes,UF,Municipio,Endereco_Completo,Endereco_Padronizado,CEP,Telefone,E-mail,Site,Dirigente,Fonte,Ano_Criacao
724,iftm_polo_de_inovacao_uberaba,IFTM,Instituto Federal do Triângulo Mineiro,Polo de Inovação Uberaba,Sudeste,MG,Uberaba,NaN,Uberaba/MG,NaN,NaN,NaN,NaN,NaN,NaN,2021
725,ifpb_polo_de_inovacao_joao_pessoa,IFPB,Instituto Federal da Paraíba,Polo de Inovação João Pessoa,Nordeste,PB,Coronel João Pessoa,NaN,Coronel João Pessoa/PB,NaN,NaN,NaN,NaN,NaN,NaN,2018
726,ifpb_campus_avancado_areia,IFPB,Instituto Federal da Paraíba,Campus Avançado Areia,Nordeste,PB,Areia,NaN,Areia/PB,NaN,NaN,NaN,NaN,NaN,NaN,2020
727,ifpe_campus_abreu_e_lima,IFPE,Instituto Federal de Pernambuco,Campus Abreu e Lima,Nordeste,PE,Abreu e Lima,NaN,Abreu e Lima/PE,NaN,NaN,NaN,NaN,NaN,NaN,2017
728,cefet_rj_unidade_maracana,CEFET-RJ,Centro Federal de Educação Tecnológica Celso S...,Unidade Maracanã,Sudeste,RJ,Rio de Janeiro,NaN,Rio de Janeiro/RJ,NaN,NaN,NaN,NaN,NaN,NaN,1917
729,iff_campus_bom_jesus_do_itabapoana,IFF,Instituto Federal Fluminense,Campus Bom Jesus do Itabapoana,Sudeste,RJ,Bom Jesus do Itabapoana,NaN,Bom Jesus do Itabapoana/RJ,NaN,NaN,NaN,NaN,NaN,NaN,1970
730,ifrn_campus_avancado_jucurutu,IFRN,Instituto Federal do Rio Grande do Norte,Campus Avançado Jucurutu,Nordeste,RN,Jucurutu,NaN,Jucurutu/RN,NaN,NaN,NaN,NaN,NaN,NaN,2018
731,if_farroupilha_campus_frederico_westphalen,IF Farroupilha,Instituto Federal Farroupilha,Campus Frederico Westphalen,Sul,RS,Frederico Westphalen,NaN,Frederico Westphalen/RS,NaN,NaN,NaN,NaN,NaN,NaN,1957
732,ifsc_polo_de_inovacao_florianopolis,IFSC,Instituto Federal de Santa Catarina,Polo de Inovação Florianópolis,Sul,SC,Florianópolis,NaN,Florianópolis/SC,NaN,NaN,NaN,NaN,NaN,NaN,2018
733,ifs_campus_poco_redondo,IFS,Instituto Federal de Sergipe,Campus Poço Redondo,Nordeste,SE,Poço Redondo,NaN,Poço Redondo/SE,NaN,NaN,NaN,NaN,NaN,NaN,2020


In [8]:
# --- Correção de valores em colunas específicas ---

def corrigir_divergencias_pelo_mapa(df_alvo: pd.DataFrame, df_referencia: pd.DataFrame, colunas_para_validar: list) -> pd.DataFrame:
    """
    Compara e corrige valores em colunas específicas do df_alvo usando o df_referencia como fonte da verdade,
    baseado na chave única 'UID'.

    Args:
        df_alvo: O DataFrame a ser corrigido (ex: df_web_scraping_RFEPT).
        df_referencia: O DataFrame com os dados corretos (ex: df_mapa_RFEPT).
        colunas_para_validar: Lista de colunas a serem verificadas (ex: ['Regioes', 'UF', 'Municipio']).

    Returns:
        O DataFrame df_alvo com os valores corrigidos.
    """
    logging.info("Iniciando verificação e correção de divergências com base no Mapa MEC...")

    # Cria uma cópia para evitar alertas de SettingWithCopy
    df_corrigido = df_alvo.copy()

    # Garante que UID seja string em ambos
    df_corrigido['UID'] = df_corrigido['UID'].astype(str)
    df_referencia['UID'] = df_referencia['UID'].astype(str)

    # Cria um dicionário de referência para acesso rápido.
    # Chave: UID, Valor: Linha do df_referencia (como dict)
    referencia_dict = df_referencia.set_index('UID').to_dict('index')

    total_correcoes = 0
    detalhes_correcoes = []

    for index, row in df_corrigido.iterrows():
        uid = row['UID']

        # Se o UID não existe na referência, pula
        if uid not in referencia_dict:
            continue

        dados_corretos = referencia_dict[uid]

        for col in colunas_para_validar:
            # Pula se a coluna não existir em algum dos dataframes
            if col not in row or col not in dados_corretos:
                continue

            valor_atual = str(row[col]).strip() if pd.notnull(row[col]) else ""
            valor_correto = str(dados_corretos[col]).strip() if pd.notnull(dados_corretos[col]) else ""

            # Normalização básica para comparação (case insensitive)
            if valor_atual.lower() != valor_correto.lower():
                # Atualiza o valor no DataFrame alvo
                df_corrigido.at[index, col] = valor_correto

                detalhes_correcoes.append({
                    'UID': uid,
                    'Coluna': col,
                    'Valor Antigo': valor_atual,
                    'Valor Novo': valor_correto
                })
                total_correcoes += 1

    if total_correcoes > 0:
        logging.info(f"Foram realizadas {total_correcoes} correções em {len(detalhes_correcoes)} registros.")
        # Opcional: Mostrar amostra das correções
        df_log_correcoes = pd.DataFrame(detalhes_correcoes)
        print("\n" + "="*80)
        print("AMOSTRA DE CORREÇÕES REALIZADAS (DIVERGÊNCIAS ENCONTRADAS)")
        print("="*80)
        display(df_log_correcoes.head(10))
    else:
        logging.info("Nenhuma divergência encontrada nas colunas especificadas.")

    return df_corrigido

# --- COMO EXECUTAR (Insira isso após carregar ambos os dataframes e criar os UIDs) ---

colunas_alvo = ['Regioes', 'UF', 'Municipio']

if 'df_web_scraping_RFEPT' in locals() and 'df_mapa_RFEPT' in locals():
    df_web_scraping_RFEPT = corrigir_divergencias_pelo_mapa(
        df_web_scraping_RFEPT,
        df_mapa_RFEPT,
        colunas_alvo
    )


AMOSTRA DE CORREÇÕES REALIZADAS (DIVERGÊNCIAS ENCONTRADAS)


,UID,Coluna,Valor Antigo,Valor Novo
0,ifsuldeminas_campus_avancado_carmo_de_minas,Municipio,Bairro Chacrinha Carmo de Minas,Carmo de Minas
1,ifsuldeminas_campus_muzambinho,Municipio,000 Muzambinho,Muzambinho
2,ifsuldeminas_campus_passos,Municipio,Bairro Penha II Passos,Passos
3,ifsuldeminas_campus_pouso_alegre,Municipio,Bairro Parque Real Pouso Alegre,Pouso Alegre
4,cefet_mg_uned_divinopolis,Municipio,,Divinópolis
5,ifmg_campus_bambui,Municipio,Km 05 Bambuí,Bambuí
6,ifmg_campus_betim,Municipio,Bairro São Caetano Betim,Betim
7,ifmg_campus_ouro_preto,Municipio,Morro do Cruzeiro Ouro Preto,Ouro Preto
8,ifmg_campus_ribeirao_das_neves,Municipio,Vila Esplanada Ribeirão das Neves,Ribeirão das Neves
9,ifmg_campus_sabara,Municipio,Sobradinho Sabará,Sabará


In [9]:
# ==============================================================================
# @title **SEÇÃO 7: CRIAÇÃO DA COLUNA 'Tipo_Unidade'**
# ==============================================================================

def definir_tipo_unidade(campus_text: str) -> str:
    """
    Categoriza a unidade com base no texto da coluna 'Campus_IF'.
    A ordem de verificação é importante: termos mais específicos (com mais palavras)
    devem ser verificados antes de termos genéricos.
    """
    if not isinstance(campus_text, str) or not campus_text:
        return 'Não Identificado'

    # Normaliza para comparação (remove acentos e coloca em minúsculas)
    # Ex: "Câmpus" vira "campus", "Pólo" vira "polo"
    texto_norm = unidecode(campus_text).lower()

    # Dicionário de Mapeamento (Ordem de prioridade é CRUCIAL)
    # Chave: termo a buscar no texto normalizado
    # Valor: Categoria final bonitinha
    mapeamento = [
        ("reitoria", "Reitoria"),
        ("campus avancado", "Campus Avançado"), # Verificar antes de 'campus'
        ("campus", "Campus"),
        ("centro de referencia", "Centro de Referência"),
        ("polo de inovacao", "Polo de Inovação"),
        ("parque tecnologico", "Parque Tecnológico"),
        ("uned", "Uned"),
        ("unidade", "Unidade"),
        # Escolas
        ("escola agricola", "Escola Agrícola"),
        ("escola de enfermagem", "Escola de Enfermagem"),
        ("escola de saude", "Escola de Saúde"),
        ("escola de musica", "Escola de Música"),
        # Colégios
        ("colegio agricola", "Colégio Agrícola"),
        ("colegio tecnico", "Colégio Técnico"),
        ("colegio politecnico", "Colégio Politécnico"),
        ("colegio universitario", "Colégio Universitário"),
        # Outros
        ("universidade tecnologica", "Universidade Tecnológica"),
        ("centro de educacao profissional", "Centro de Educação Profissional"),
        ("centro de ensino", "Centro de Ensino")
    ]

    for termo_busca, categoria in mapeamento:
        if termo_busca in texto_norm:
            return categoria

    return "Outros"

# --- Aplicação no DataFrame ---

if 'df_web_scraping_RFEPT' in locals() and not df_web_scraping_RFEPT.empty:
    logging.info("Iniciando a criação da coluna 'Tipo_Unidade'...")

    # Aplica a função linha a linha
    df_web_scraping_RFEPT['Tipo_Unidade'] = df_web_scraping_RFEPT['Campus_IF'].apply(definir_tipo_unidade)

    logging.info("Coluna 'Tipo_Unidade' criada com sucesso.")

    # Reordenar para colocar a nova coluna perto de Campus_IF
    cols = list(df_web_scraping_RFEPT.columns)
    if 'Tipo_Unidade' in cols and 'Campus_IF' in cols:
        cols.pop(cols.index('Tipo_Unidade'))
        idx_campus = df_web_scraping_RFEPT.columns.get_loc('Campus_IF')
        cols.insert(idx_campus + 1, 'Tipo_Unidade')
        df_web_scraping_RFEPT = df_web_scraping_RFEPT[cols]

    # Exibição dos resultados
    print("\n" + "="*80)
    print("CONTAGEM DE UNIDADES POR TIPO")
    print("="*80)
    print(df_web_scraping_RFEPT['Tipo_Unidade'].value_counts())

    print("\n" + "="*80)
    print("AMOSTRA DOS DADOS COM A NOVA COLUNA")
    print("="*80)
    # Filtra colunas principais para visualização
    cols_view = ['Nome_IF', 'Campus_IF', 'Tipo_Unidade']
    display(df_web_scraping_RFEPT[cols_view].head(10))

else:
    logging.warning("DataFrame 'df_web_scraping_RFEPT' não encontrado.")


CONTAGEM DE UNIDADES POR TIPO
Tipo_Unidade
Campus                             594
Campus Avançado                     46
Reitoria                            41
Uned                                15
Polo de Inovação                    13
Outros                               6
Colégio Técnico                      6
Colégio Agrícola                     3
Unidade                              2
Escola de Música                     2
Centro de Educação Profissional      1
Centro de Referência                 1
Centro de Ensino                     1
Escola Agrícola                      1
Escola de Saúde                      1
Colégio Politécnico                  1
Name: count, dtype: int64

AMOSTRA DOS DADOS COM A NOVA COLUNA


,Nome_IF,Campus_IF,Tipo_Unidade
0,Instituto Federal do Sul de Minas Gerais,Reitoria do Instituto Federal do Sul de Minas ...,Reitoria
1,Instituto Federal do Sul de Minas Gerais,Campus Avançado Carmo de Minas,Campus Avançado
2,Instituto Federal do Sul de Minas Gerais,Campus Avançado Três Corações,Campus Avançado
3,Instituto Federal do Sul de Minas Gerais,Campus Inconfidentes,Campus
4,Instituto Federal do Sul de Minas Gerais,Campus Machado,Campus
5,Instituto Federal do Sul de Minas Gerais,Campus Muzambinho,Campus
6,Instituto Federal do Sul de Minas Gerais,Campus Passos,Campus
7,Instituto Federal do Sul de Minas Gerais,Campus Poços de Caldas,Campus
8,Instituto Federal do Sul de Minas Gerais,Campus Pouso Alegre,Campus
9,Centro Federal de Educação Tecnológica de Mina...,Reitoria do Centro Federal de Educação Tecnoló...,Reitoria


In [10]:
# ==============================================================================
# @title **SEÇÃO 8: PÓS-PROCESSAMENTO - ANÁLISE DE TAMANHO DAS STRINGS**
# ==============================================================================
# Esta seção analisa o tamanho das strings em colunas específicas para entender
# a variabilidade e identificar possíveis anomalias nos dados extraídos e limpos.
# ==============================================================================

# Lista das colunas a serem analisadas
colunas_para_analise = [
    'UID',
    'Nome_IF',
    'Campus_IF',
    'Endereco_Completo',
    'Endereco_Padronizado',
    'Telefone',
    'E-mail',
    'Site',
    'Dirigente'
]

print("\n" + "="*80)
print("ANÁLISE DO NÚMERO DE CARACTERES POR LINHA")
print("="*80)

# Itera sobre cada coluna e calcula as estatísticas
for coluna in colunas_para_analise:
    if coluna in df_web_scraping_RFEPT.columns:
        # Calcula o número de caracteres para cada linha, tratando valores nulos
        # Usamos .astype(str) para garantir que todos os valores sejam strings antes de calcular o comprimento
        # Substituímos 'None' por '' para que o comprimento de valores nulos seja 0
        tamanhos = df_web_scraping_RFEPT[coluna].astype(str).replace('None', '').apply(len)

        if not tamanhos.empty:
            min_chars = tamanhos.min()
            max_chars = tamanhos.max()
            media_chars = tamanhos.mean()
            mediana_chars = tamanhos.median()

            print(f"\n--- Coluna: '{coluna}' ---")
            print(f"  Mínimo de caracteres: {min_chars}")
            print(f"  Máximo de caracteres: {max_chars}")
            print(f"  Média de caracteres: {media_chars:.2f}")
            print(f"  Mediana de caracteres: {mediana_chars:.2f}")
        else:
            print(f"\n--- Coluna: '{coluna}' ---")
            print("  Não há dados para analisar nesta coluna.")
    else:
        print(f"\n--- Coluna: '{coluna}' ---")
        print("  Coluna não encontrada no DataFrame.")

print("\n" + "="*80)
print("Fim da análise de tamanho das strings.")
print("="*80)


ANÁLISE DO NÚMERO DE CARACTERES POR LINHA

--- Coluna: 'UID' ---
  Mínimo de caracteres: 15
  Máximo de caracteres: 88
  Média de caracteres: 27.29
  Mediana de caracteres: 23.50

--- Coluna: 'Nome_IF' ---
  Mínimo de caracteres: 16
  Máximo de caracteres: 86
  Média de caracteres: 32.30
  Mediana de caracteres: 29.00

--- Coluna: 'Campus_IF' ---
  Mínimo de caracteres: 10
  Máximo de caracteres: 86
  Média de caracteres: 21.56
  Mediana de caracteres: 17.00

--- Coluna: 'Endereco_Completo' ---
  Mínimo de caracteres: 0
  Máximo de caracteres: 163
  Média de caracteres: 73.03
  Mediana de caracteres: 72.50

--- Coluna: 'Endereco_Padronizado' ---
  Mínimo de caracteres: 0
  Máximo de caracteres: 157
  Média de caracteres: 68.51
  Mediana de caracteres: 67.00

--- Coluna: 'Telefone' ---
  Mínimo de caracteres: 0
  Máximo de caracteres: 32
  Média de caracteres: 13.72
  Mediana de caracteres: 14.00

--- Coluna: 'E-mail' ---
  Mínimo de caracteres: 0
  Máximo de caracteres: 44
  Média de 

In [11]:
df_web_scraping_RFEPT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 734 entries, 0 to 733
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   UID                   734 non-null    object
 1   Sigla_IF              733 non-null    object
 2   Nome_IF               734 non-null    object
 3   Campus_IF             734 non-null    object
 4   Tipo_Unidade          734 non-null    object
 5   Regioes               733 non-null    object
 6   UF                    734 non-null    object
 7   Municipio             723 non-null    object
 8   Endereco_Completo     708 non-null    object
 9   Endereco_Padronizado  727 non-null    object
 10  CEP                   715 non-null    object
 11  Telefone              715 non-null    object
 12  E-mail                715 non-null    object
 13  Site                  715 non-null    object
 14  Dirigente             715 non-null    object
 15  Fonte                 715 non-null    ob

In [12]:
# ==============================================================================
# @title **SEÇÃO 7: EXPORTAR PARA O GOOGLE SHEETS `df_web_scraping_RFEPT`**
# ==============================================================================

# ======================== Configurações da Planilha ========================
# ID da planilha
ID_PLANILHA = '18S79OGp2RV5QmxO3Rb39PCmRkY_WrCL3gWgRGqIPELk' # Planilha pública
# Nome da aba, se não existir, será criada.
NOME_ABA = 'Web-Scraping-and-merge-RFEPT-map'
# ==============================================================

# ======================== Configuração das Colunas para Exportar ========================
# Defina a lista de colunas na ordem desejada para exportação.
# Certifique-se de que os nomes das colunas correspondem exatamente aos do DataFrame.
colunas_para_exportar = [
    'UID',
    'Ano_Criacao',
    'Sigla_IF',
    'Nome_IF',
    'Campus_IF',
    'Tipo_Unidade',
    'Regioes',
    'UF',
    'Municipio',
    'Endereco_Padronizado', # Usando o endereço padronizado para geocodificação
    'CEP',
    'Telefone',
    'E-mail',
    'Site',
    'Dirigente',
    'Fonte'
    # Adicione 'Latitude' e 'Longitude' aqui se a seção de georreferenciamento for bem-sucedida
    # 'Latitude',
    # 'Longitude'
]
# ========================================================================================


# 1. Autenticação do Google Colab
print("Autenticando-se no Google...")
# Inicia autenticação no Google
auth.authenticate_user()
print("Autenticação concluída.")

# 2. Inicializar o cliente gspread
# Utilizar as credenciais padrão estabelecidas pelo auth.authenticate_user()
credentials, project = google.auth.default()
gc = Client(auth=credentials)

try:
    # 3. Abrir a planilha usando o ID (Chave)
    sh = gc.open_by_key(ID_PLANILHA)
    print(f"Planilha ID: '{ID_PLANILHA}' acessada com sucesso.")

    # 4. Selecionar ou criar a aba (worksheet)
    try:
        # Tenta selecionar a aba existente
        worksheet = sh.worksheet(NOME_ABA)
        # Limpa o conteúdo existente para substituí-lo
        worksheet.clear()
        print(f"Aba '{NOME_ABA}' limpa e pronta para receber os novos dados.")
    except gspread.exceptions.WorksheetNotFound:
        # Se não encontrar, adiciona uma nova aba
        # Adicionamos um número grande de linhas/colunas para garantir espaço
        worksheet = sh.add_worksheet(title=NOME_ABA, rows="1000", cols="20")
        print(f"Aba '{NOME_ABA}' criada.")


    # 5. Preparar os dados para exportação, selecionando e reordenando as colunas
    # Certifica-se que df_web_scraping_RFEPT está definido e não vazio
    if 'df_web_scraping_RFEPT' in locals() and not df_web_scraping_RFEPT.empty:
        # Seleciona apenas as colunas desejadas na ordem especificada
        df_export =df_web_scraping_RFEPT[colunas_para_exportar].copy() # Use .copy() para evitar SettingWithCopyWarning

        # Replace pd.NA with None for JSON serialization
        df_export_cleaned = df_export.replace({pd.NA: None})

        # Converte o DataFrame para uma lista de listas (formato que o gspread usa)
        # Inclui o cabeçalho (nomes das colunas)
        dados_para_sheet = [df_export_cleaned.columns.tolist()] + df_export_cleaned.values.tolist()
    else:
        raise ValueError("DataFrame 'df_web_scraping_RFEPT' não encontrado ou está vazio.")

    # 6. Exportar os dados (atualiza toda a faixa a partir de A1)
    # Usar argumentos nomeados para evitar o DeprecationWarning
    worksheet.update(values=dados_para_sheet, range_name='A1')

    # 7. Exibir o link
    print("-" * 50)
    print("Exportação concluída com sucesso!")
    print(f"Acesse a planilha aqui: {sh.url}")
    print("-" * 50)

except gspread.exceptions.SpreadsheetNotFound:
    print("-" * 50)
    print(f"ERRO: Planilha com ID '{ID_PLANILHA}' não encontrada.")
    print("Verifique se o ID está correto ou se você possui acesso a ela.")
    print("-" * 50)
except gspread.exceptions.APIError as e:
    print("-" * 50)
    print("ERRO de API (Permissão):")
    print("Verifique se a conta Google usada na autenticação tem permissão de 'Editor' na planilha.")
    print(f"Detalhes do erro: {e}")
    print("-" * 50)
except KeyError as e:
    print("-" * 50)
    print(f"ERRO: Coluna '{e}' não encontrada no DataFrame.")
    print("Verifique a lista 'colunas_para_exportar' na SEÇÃO 7.")
    print("-" * 50)
except Exception as e:
    # Captura outros erros inesperados
    print(f"Ocorreu um erro inesperado durante a exportação: {e}")
    print("-" * 50)

Autenticando-se no Google...
Autenticação concluída.
Planilha ID: '18S79OGp2RV5QmxO3Rb39PCmRkY_WrCL3gWgRGqIPELk' acessada com sucesso.
Aba 'Web-Scraping-and-merge-RFEPT-map' criada.
--------------------------------------------------
Exportação concluída com sucesso!
Acesse a planilha aqui: https://docs.google.com/spreadsheets/d/18S79OGp2RV5QmxO3Rb39PCmRkY_WrCL3gWgRGqIPELk
--------------------------------------------------
